In [ ]:
!pip install pybaseball
!pip install catboost
!pip install tensorflow

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pybaseball import statcast, team_game_logs, retrosheet, playerid_lookup
import pybaseball
from google.colab import files

from sklearn.model_selection import cross_val_predict, cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, r2_score, log_loss
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

import arviz as az
import pymc as pm
import pickle

pl.Config(tbl_cols=1000)
pl.Config(tbl_rows=15)

# Data Collecting

In [ ]:
data_2021 = statcast('2021-04-01', '2021-10-03')
data_2021.to_csv('df_2021.csv')
files.download('df_2021.csv')

In [ ]:
data_2022 = statcast('2022-04-07', '2022-10-02')
data_2022.to_csv('df_2022.csv')
files.download('df_2022.csv')

In [ ]:
data_2023 = statcast('2023-03-30', '2023-10-01')
data_2023.to_csv('data_2023.csv')
files.download('data_2023.csv')

In [ ]:
data_2024 = statcast('2024-03-28', '2024-09-29')
data_2024.to_csv('df_2024.csv')
files.download('df_2024.csv')

In [ ]:
data_2021 = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_2021.csv')
data_2022 = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_2022.csv')
data_2023 = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/data_2023.csv')
data_2024 = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_2024.csv')

data = pl.concat([data_2021, data_2022, data_2023, data_2024], how='vertical_relaxed')

# get year
data = data.with_columns(
    pl.col('game_date').str.strptime(pl.Date, format="%Y-%m-%d").dt.year().alias('year')
)

In [ ]:
data.write_csv('data_2021_2024.csv')
files.download('data_2021_2024.csv')

# Feature Engineering and Data Cleaning

In [ ]:
data_2021_2024_features = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/data_2021_2024.csv')

In [ ]:
# Filter weird pitches
data_2021_2024_features = data_2021_2024_features.filter(~pl.col('pitch_type').is_in(['CS', 'PO', 'EP', 'KN', 'FA',]))
# Filter weird events
data_2021_2024_features = data_2021_2024_features.filter(
    ((~pl.col('events').is_in(['catcher_interf', 'sac_bunt', 'sac_bunt_double_play', 'truncated_pa'])) |
     (pl.col('events').is_null())
    ) &
    ((~pl.col('description').is_in(['missed_bunt', 'foul_bunt', 'bunt_foul_tip'])) |
     (pl.col('description').is_null())
    )
)

# Create event targets
data_2021_2024_features = data_2021_2024_features.with_columns(
    pl.when(pl.col('p_throws') == 'R')
    .then(1)
    .otherwise(0).alias('righty'),

    pl.when(pl.col('description') == 'foul')
    .then(pl.col('description'))
    .otherwise(pl.col('bb_type')).alias('bb_type'),

    swing =
    pl.when(
        pl.col('description').is_in(['blocked_ball', 'ball', 'hit_by_pitch', 'called_strike'])
    )
    .then(0)
    .otherwise(1),

    called_strike =
    pl.when(
        pl.col('description').is_in(['called_strike'])
    )
    .then(1)
    .otherwise(0),

    called_ball =
    pl.when(
        pl.col('description').is_in(['blocked_ball', 'ball'])
    )
    .then(1)
    .otherwise(0),

    called_hbp =
    pl.when(
        pl.col('description').is_in(['hit_by_pitch'])
    )
    .then(1)
    .otherwise(0),

    whiff =
    pl.when(
        pl.col('description').is_in(['swinging_strike', 'swinging_strike_blocked'])
    )
    .then(1)
    .otherwise(0),

    contact =
    pl.when(
        pl.col('description').is_in(['foul', 'hit_into_play'])
    )
    .then(1)
    .otherwise(0),

    foul =
    pl.when(
        pl.col('description').is_in(['foul'])
    )
    .then(1)
    .otherwise(0),

    out =
    pl.when(
        pl.col('events').is_in(['sac_fly', 'sac_fly_double_play', 'field_error', 'force_out', 'field_out',
                                'fielders_choice_out', 'double_play', 'fielders_choice', 'triple_play',
                                'strikeout_double_play', 'grounded_into_double_play'])
    )
    .then(1)
    .otherwise(0),

    single = pl.when(pl.col('events') == 'single').then(1).otherwise(0),

    double = pl.when(pl.col('events') == 'double').then(1).otherwise(0),

    triple = pl.when(pl.col('events') == 'triple').then(1).otherwise(0),

    home_run = pl.when(pl.col('events') == 'home_run').then(1).otherwise(0),

    ground_ball = pl.when(pl.col('bb_type') == 'ground_ball').then(1).otherwise(0),

    line_drive = pl.when(pl.col('bb_type') == 'line_drive').then(1).otherwise(0),

    fly_ball = pl.when(pl.col('bb_type') == 'fly_ball').then(1).otherwise(0),

    popup = pl.when(pl.col('bb_type') == 'popup').then(1).otherwise(0)

)

In [ ]:
# adjust lefties horz
data_2021_2024_features = data_2021_2024_features.with_columns(
    pl.when(pl.col('p_throws') == 'L')
    .then(pl.col('release_pos_x')*-1)
    .otherwise('release_pos_x')
    .alias('adj_release_pos_x'),

    pl.when(pl.col('p_throws') == 'L')
    .then(pl.col('pfx_x')*-1)
    .otherwise('pfx_x')
    .alias('adj_pfx_x'),

    pl.when(pl.col('p_throws')=='L')
    .then(360 - pl.col('spin_axis'))
    .otherwise(pl.col('spin_axis'))
    .alias('adj_spin_axis'),

    pl.when(pl.col('p_throws')=='L')
    .then(pl.col('vx0')*-1)
    .otherwise(pl.col('vx0'))
    .alias('adj_vx0'),

    pl.when(pl.col('p_throws')=='L')
    .then(pl.col('ax')*-1)
    .otherwise(pl.col('ax'))
    .alias('adj_ax')
)

# get most thrown fastball
pitch_count = (
    data_2021_2024_features
    .filter(pl.col('pitch_type').is_in(['FF', 'SI', 'FC']))
    .select('pitch_type', 'pitcher')
    .group_by('pitcher')
    .agg(
        pl.col('pitch_type')
        .mode()
        .first()
        .alias('primary_fastball')
    )
)
data_2021_2024_features = data_2021_2024_features.join(pitch_count, on='pitcher', how='left')

# get avg velo of most thrown fastball
fb_velo = (
    data_2021_2024_features
    .filter(pl.col('pitch_type') == pl.col('primary_fastball'))
    .select('pitcher', 'release_speed')
    .group_by('pitcher')
    .agg(
        pl.col('release_speed')
        .mean()
        .alias('primary_fb_speed')
    )
)
data_2021_2024_features = data_2021_2024_features.join(fb_velo, on='pitcher', how='left')

# get avg vert of most thrown fastball
fb_vert = (
    data_2021_2024_features
    .filter(pl.col('pitch_type') == pl.col('primary_fastball'))
    .select('pitcher', 'pfx_z')
    .group_by('pitcher')
    .agg(
        pl.col('pfx_z')
        .mean()
        .alias('primary_fb_vert')
    )
)
data_2021_2024_features = data_2021_2024_features.join(fb_vert, on='pitcher', how='left')

# get avg horz of most thrown fastball
fb_horz = (
    data_2021_2024_features
    .filter(pl.col('pitch_type') == pl.col('primary_fastball'))
    .select('pitcher', 'pfx_x')
    .group_by('pitcher')
    .agg(
        pl.col('pfx_x')
        .mean()
        .alias('primary_fb_horz')
    )
)
data_2021_2024_features = data_2021_2024_features.join(fb_horz, on='pitcher', how='left')

# get secondary pitch differences
data_2021_2024_features = data_2021_2024_features.with_columns(
    secondary_velo_differential = pl.when(pl.col('pitch_type').is_in(['FF', 'FC', 'SI']))
    .then(0)
    .otherwise(pl.col('primary_fb_speed') - pl.col('release_speed')),

    secondary_vert_differential = pl.when(pl.col('pitch_type').is_in(['FF', 'FC', 'SI']))
    .then(0)
    .otherwise(pl.col('primary_fb_vert') - pl.col('pfx_z')),

    secondary_horz_differential = pl.when(pl.col('pitch_type').is_in(['FF', 'FC', 'SI']))
    .then(0)
    .otherwise(pl.col('primary_fb_horz') - pl.col('pfx_x'))
)

# get same side
data_2021_2024_features = data_2021_2024_features.with_columns(
    pl.when(pl.col('p_throws')==pl.col('stand'))
    .then(1)
    .otherwise(0)
    .alias('same_side')
)

## Dropping Nulls

In [ ]:
# relevant feats
data_2021_2024_features = df.select('game_year', 'game_date', 'player_name', 'batter', 'pitcher',
               'pitch_type', 'events', 'description', 'bb_type', 'stand', 'p_throws', 'same_side', 'balls', 'strikes',
               'release_pos_x', 'adj_release_pos_x', 'release_pos_z', 'release_pos_y', 'release_extension', 'arm_angle',
               'release_speed', 'release_spin_rate', 'spin_axis', 'adj_spin_axis', 'pfx_x', 'adj_pfx_x', 'pfx_z', 'api_break_z_with_gravity', 'api_break_x_arm',
               'primary_fb_speed', 'primary_fb_vert', 'primary_fb_horz', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential',
               'plate_x', 'plate_z', 'sz_top', 'sz_bot',
               'vx0', 'adj_vx0', 'vy0', 'vz0', 'ax', 'adj_ax', 'ay', 'az',
               'launch_speed', 'launch_angle', 'estimated_woba_using_speedangle', 'woba_value', 'delta_run_exp', 'delta_home_win_exp')

# drop nulls
data_2021_2024_features = data_2021_2024_features.drop_nulls(
    subset = ['pitch_type', 'release_pos_x', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'api_break_z_with_gravity', 'primary_fb_speed', 'delta_run_exp']
)

# filter incomplete exit velo/launch angle data
data_2021_2024_features = data_2021_2024_features.filter(
    ((pl.col('description') != 'hit_into_play')) |
    ((pl.col('description') == 'hit_into_play') &
        (
        (pl.col("launch_speed").is_not_null()) &
        (pl.col("launch_angle").is_not_null()) &
        (pl.col('bb_type').is_not_null())
        )
    )
)

In [ ]:
data_2021_2024_features.null_count()

In [ ]:
data_2021_2024_features.shape

In [ ]:
data_2021_2024_features.write_csv('data_2021_2024_features.csv')
files.download('data_2021_2024_features.csv')

## For model correlation validation

In [ ]:
# query relevant performance metrics
corr_df = pl.DataFrame(pybaseball.pitching_stats(start_season = 2021, end_season = 2024, qual = 100)).select('Season', 'Name', 'IDfg', 'IP', 'ERA', 'FIP', 'xFIP', 'SIERA', 'botStf', 'Stuff+')

# match name format to statcast for joining
corr_df = corr_df.with_columns(
        pl.col('Name').str.split(" ").map_elements(lambda parts: f"{parts[-1]}, {' '.join(parts[:-1])}").alias('names')
        )

# next season performance
corr_df1 = corr_df.select([
    pl.col("Name"),
    (pl.col('Season') - 1).alias('Season'),
    pl.col("ERA").alias("ERA_next"),
    pl.col("FIP").alias("FIP_next"),
    pl.col("xFIP").alias("xFIP_next"),
    pl.col("SIERA").alias("SIERA_next"),
])

# joining next season to this season
corr_df = corr_df.join(corr_df1, on=['Name', 'Season'], how= 'left')
corr_df = corr_df.drop_nulls()

In [ ]:
corr_df.write_csv('corr_df.csv')
files.download('corr_df.csv')

# Baseline Models (XGBoost)

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/data_2021_2024_features.csv')

In [ ]:
df.head()

In [ ]:
def validation_plot(df, actuals, preds, model_name):
    r2 = (r2_score(df.select(actuals), df.select(preds)))
    rmse = np.sqrt(mean_squared_error(df.select(actuals), df.select(preds)))

    sns.regplot(data=df, x = preds, y = actuals)
    plt.xlabel('Preds')
    plt.ylabel('Actual')
    plt.title(f'{model_name} Validation \nRMSE: {round(rmse, 4)}, R-Squared: {round(r2, 4)}')
    plt.xlim([-3, 4])
    plt.ylim([-3, 4])
    plt.show()

In [ ]:
corr_df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/corr_df.csv')
def corr_valid(df = df, metric = None, model = ""):
    corr_df1 = corr_df.clone()
    # get mean stuff in year
    mean_stuff = df.group_by(pl.col('game_year'), pl.col('player_name')).agg(pl.col(metric).mean())
    # join to performance corr df
    corr_df1 = corr_df1.join(mean_stuff, left_on = ['Season', 'names'], right_on = ['game_year', 'player_name'], how='inner')
    y_vars = ['ERA_next', 'FIP_next', 'xFIP_next', 'SIERA_next']
    x_vars = ['ERA', 'FIP', 'xFIP', 'SIERA', 'botStf', 'Stuff+', metric]

    correlations = []

    # get correlations for each metric
    for x in x_vars:
        row = []
        for y in y_vars:
            corr = corr_df1.select(pl.corr(x, y)).item()
            row.append(abs(corr))
        correlations.append(row)

    # corr df for heatmap
    corr_pd = pd.DataFrame((correlations), index=x_vars, columns=y_vars)

    # plot heatmap
    sns.heatmap(corr_pd, annot=True, vmin = 0.2, vmax = 0.65, cmap="coolwarm", fmt = '.2f')
    plt.title(f"{model} Correlation Matrix (Min 100 IP)")
    plt.show()

## CatBoost

In [ ]:
# catboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'delta_run_exp'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = CatBoostRegressor(
    cat_features = ['righty'],
    loss_function = 'RMSE'
)
preds = cross_val_predict(model, X, y , cv=5, method='predict')

df = df.with_columns(pl.Series("run_exp_pred_catboost", preds))

In [ ]:
validation_plot(df = df, actuals = 'delta_run_exp', preds = 'run_exp_pred_catboost', model_name = 'CatBoost')

In [ ]:
corr_valid(metric = 'run_exp_pred_catboost', model = 'CatBoost')

## XGBoost

In [ ]:
# xgboost model
df = df.with_columns(
    righty = pl.when(pl.col('p_throws') == 'R')
    .then(1)
    .otherwise(0)
)
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'delta_run_exp'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBRegressor()
preds = cross_val_predict(model, X, y , cv=5, method='predict')

df = df.with_columns(pl.Series("run_exp_pred_xgb", preds))

In [ ]:
validation_plot(df = df, actuals = 'delta_run_exp', preds = 'run_exp_pred_xgb', model_name = 'XGBoost')

In [ ]:
corr_valid(metric = 'run_exp_pred_xgb', model = 'XGBoost')

In [ ]:
# rv pred distribution
sns.histplot(df.select(pl.col('run_exp_pred_xgb')))
plt.xlim([-0.05, 0.05])
plt.show()

In [ ]:
# best stuff by year
df.group_by(pl.col('game_year'), pl.col('player_name')).agg(pl.col('run_exp_pred_xgb').mean()).sort('run_exp_pred_xgb', descending = False).head(10)

In [ ]:
# best pitches by year
(df.with_columns([
        pl.len().over(["player_name", "pitch_type"]).alias("pitch_count")
    ])
    .filter(pl.col("pitch_count") >= 100)
    .group_by(pl.col('game_year'), pl.col('player_name'), pl.col('pitch_type'))
    .agg(pl.col('run_exp_pred_xgb').mean())
    .sort('run_exp_pred_xgb', descending = False).head(10)
)

In [ ]:
df.write_csv('df1.csv')
files.download('df1.csv')

## LGBM

In [ ]:
# lgbm model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'delta_run_exp'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = LGBMRegressor()
preds = cross_val_predict(model, X, y , cv=5, method='predict')

df = df.with_columns(pl.Series("run_exp_pred_lgbm", preds))

In [ ]:
validation_plot(df = df, actuals = 'delta_run_exp', preds = 'run_exp_pred_lgbm', model_name = 'LGBM')

In [ ]:
corr_valid(metric = 'run_exp_pred_lgbm', model = 'LGBM')

## Neural Net 2 Hidden Layers

In [ ]:
# Neural net 3 layers
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'delta_run_exp'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

y_pred_all = np.zeros_like(y)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Dense(64, activation = 'relu', input_shape = (X.shape[1], )),
        Dense(32, activation = 'relu'),
        Dense(1)
    ])
    model.compile(optimizer = Adam(), loss = 'mse')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, batch_size = 1024, epochs = 15, callbacks = [early_stop], verbose = 1)

    y_pred = model.predict(X_test_scaled).flatten()

    y_pred_all[test_idx] = y_pred

df = df.with_columns(pl.Series("run_exp_pred_nn", y_pred_all))

In [ ]:
validation_plot(df = df, actuals = 'delta_run_exp', preds = 'run_exp_pred_nn', model_name = 'Neural Net 3 Layers')

In [ ]:
corr_valid(metric = 'run_exp_pred_nn', model = 'Neural Net 3 Layers')

## Neural Net 3 Hidden Layers

In [ ]:
# Neural net 4 layers
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'delta_run_exp'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

y_pred_all = np.zeros_like(y)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Dense(128, activation = 'relu', input_shape = (X.shape[1], )),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(1)
    ])
    model.compile(optimizer = Adam(learning_rate = 0.0005), loss = 'mse')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 2,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, batch_size = 1024, epochs = 15, callbacks = [early_stop], verbose = 1)

    y_pred = model.predict(X_test_scaled).flatten()

    y_pred_all[test_idx] = y_pred

df = df.with_columns(pl.Series("run_exp_pred_nn_4", y_pred_all))

In [ ]:
validation_plot(df = df, actuals = 'delta_run_exp', preds = 'run_exp_pred_nn_4', model_name = 'Neural Net 4 Layers')

In [ ]:
corr_valid(metric = 'run_exp_pred_nn_4', model = 'Neural Net 4 Layers')

# Swing Model (XGBoost)

In [ ]:
def calibration_plot(data = None, n_bins = 10, preds = None, actuals = None, model = None):
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,4))

    sns.histplot(data[preds], ax = ax1)
    ax1.set_xlabel('Predicted Probability')
    ax1.set_ylabel('Count')
    ax1.set_title(f'{model} \nPredicted Probability Distribution')
    ax1.set_xlim([0, 1])

    # Bin predicted probabilities
    _df = data.with_columns([
        (pl.col(preds) * 10)
        .cast(pl.Int32)
        .alias("bin")
    ])
    _df = _df.with_columns(
        bin = pl.when(pl.col('bin') > 9)
        .then(9)
        .otherwise(pl.col('bin'))
    )

    logloss = round(log_loss(_df[actuals], _df[preds]), 4)

    # Group by bin and calculate avg predicted prob and actual rate
    binned = (
        _df.group_by("bin")
        .agg([
            pl.col(preds).mean().alias("mean_pred_prob"),
            pl.col(actuals).mean().alias("empirical_prob"),
            pl.len().alias("count")
        ])
        .sort("bin")
    )

    binned_pd = binned.to_pandas()

    # Plot calibration curve
    ax2.plot(binned_pd["mean_pred_prob"], binned_pd["empirical_prob"], marker='o', label='Calibration')
    ax2.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect Calibration')
    ax2.set_xlabel('Mean Predicted Probability')
    ax2.set_ylabel('Empirical Probability')
    ax2.set_title(f'{model} Calibration Plot \nLog Loss: {logloss}')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
df['swing'].mean()

## XGBoost

In [ ]:
# xgboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'swing'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBClassifier()
preds = cross_val_predict(model, X, y , cv=5, method='predict_proba')

df = df.with_columns(pl.Series("swing_pred_xgb", preds[:, 1]),
                     pl.Series("take_pred_xgb", preds[:, 0]))

In [ ]:
calibration_plot(n_bins = 10, data = df, preds = 'swing_pred_xgb', actuals = 'swing', model = 'XGBoost Swing')

## Neural Net 2 Layers

In [ ]:
# Neural net 2 layers
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'swing'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

y_pred_all = np.zeros_like(y, dtype=float)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1], )),
        Dense(32, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ])
    model.compile(optimizer = Adam(), loss = 'binary_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, batch_size = 1024, epochs = 15, callbacks = [early_stop], verbose = 1)

    y_pred = model.predict(X_test_scaled).flatten()

    y_pred_all[test_idx] = y_pred

df = df.with_columns(pl.Series("swing_pred_nn", y_pred_all))

In [ ]:
calibration_plot(data = df, n_bins = 10, preds = 'swing_pred_nn', actuals = 'swing', model = 'Neural Net 2 Layers Swing')

# Strike/Ball/HBP Model (3 Layer DNN)

In [ ]:
# filter to relevant rows
df_take = df.filter(pl.col('swing') == 0)

df_take = df_take.with_columns([
    pl.when(pl.col('called_strike') == 1)
    .then(0)
    .when(pl.col('called_ball') ==  1)
    .then(1)
    .otherwise(2)
    .alias('take_target')
])

In [ ]:
df_take['called_strike'].mean()

In [ ]:
df_take['called_ball'].mean()

## XGBoost

In [ ]:
# xgboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'take_target'

df_pandas = df_take.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBClassifier(objective = 'multi:softprob', num_class = 3, eval_metric = 'mlogloss')
preds = cross_val_predict(model, X, y , cv=5, method='predict_proba')

df_take = df_take.with_columns(pl.Series("called_strike_pred_xgb", preds[:, 0]),
                               pl.Series("called_ball_pred_xgb", preds[:, 1]),
                               pl.Series("called_hbp_pred_xgb", preds[:, 2]))

In [ ]:
calibration_plot(data = df_take, preds = 'called_strike_pred_xgb', actuals = 'called_strike', model = 'XGBoost Called Model (Strikes)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_ball_pred_xgb', actuals = 'called_ball', model = 'XGBoost Called Model (Balls)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_hbp_pred_xgb', actuals = 'called_hbp', model = 'XGBoost Called Model (HBPs)')

## XGBoost Isotonic

In [ ]:
# xgboost isotonic model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'take_target'

df_pandas = df_take.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

strike_pred_all = np.zeros_like(y, dtype=float)
ball_pred_all = np.zeros_like(y, dtype=float)
hbp_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier(objective = 'multi:softprob', num_class = 3, eval_metric = 'mlogloss')

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'isotonic', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    strike_pred_all[test_idx] = preds[:, 0]
    ball_pred_all[test_idx] = preds[:, 1]
    hbp_pred_all[test_idx] = preds[:, 2]


df_take = df_take.with_columns(pl.Series("called_strike_pred_xgb_calibrated", strike_pred_all),
                               pl.Series("called_ball_pred_xgb_calibrated", ball_pred_all),
                               pl.Series("called_hbp_pred_xgb_calibrated", hbp_pred_all))

In [ ]:
calibration_plot(data = df_take, preds = 'called_strike_pred_xgb_calibrated', actuals = 'called_strike', model = 'XGBoost Called Model Isotonic Calibrated (Strikes)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_ball_pred_xgb_calibrated', actuals = 'called_ball', model = 'XGBoost Called Model Isotonic Calibrated (Balls)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_hbp_pred_xgb_calibrated', actuals = 'called_hbp', model = 'XGBoost Called Model Calibrated (HBP)')

## Neural Net 4 Layers

In [ ]:
# Neural net 4 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'take_target'

df_pandas = df_take.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

strike_pred_all = np.zeros_like(y, dtype=float)
ball_pred_all = np.zeros_like(y, dtype=float)
hbp_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Dense(128, activation = 'relu', input_shape = (X.shape[1],)),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(3, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    strike_pred_all[test_idx] = preds[:, 0]
    ball_pred_all[test_idx] = preds[:, 1]
    hbp_pred_all[test_idx] = preds[:, 2]


df_take = df_take.with_columns(pl.Series("called_strike_pred_nn", strike_pred_all),
                               pl.Series("called_ball_pred_nn", ball_pred_all),
                               pl.Series("called_hbp_pred_nn", hbp_pred_all))

In [ ]:
calibration_plot(data = df_take, preds = 'called_strike_pred_nn', actuals = 'called_strike', model = 'Neural Net 4 Layers Called Model (Strikes)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_ball_pred_nn', actuals = 'called_ball', model = 'Neural Net 4 Layers Called Model (Balls)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_hbp_pred_nn', actuals = 'called_hbp', model = 'Neural Net 4 Layers Called Model (HBPs)')

## Neural Net 3 Layers

In [ ]:
# Neural net 3 dense layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'take_target'

df_pandas = df_take.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

strike_pred_all = np.zeros_like(y, dtype=float)
ball_pred_all = np.zeros_like(y, dtype=float)
hbp_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(3, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    strike_pred_all[test_idx] = preds[:, 0]
    ball_pred_all[test_idx] = preds[:, 1]
    hbp_pred_all[test_idx] = preds[:, 2]


df_take = df_take.with_columns(pl.Series("called_strike_pred_nn", strike_pred_all),
                               pl.Series("called_ball_pred_nn", ball_pred_all),
                               pl.Series("called_hbp_pred_nn", hbp_pred_all))

In [ ]:
calibration_plot(data = df_take, preds = 'called_strike_pred_nn', actuals = 'called_strike', model = 'Neural Net 3 Layers Called Model (Strikes)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_ball_pred_nn', actuals = 'called_ball', model = 'Neural Net 3 Layers Called Model (Balls)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_hbp_pred_nn', actuals = 'called_hbp', model = 'Neural Net 3 Layers Called Model (HBPs)')

## Neural Net 2 Layers

In [ ]:
# Neural net 2 dense layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'take_target'

df_pandas = df_take.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

strike_pred_all = np.zeros_like(y, dtype=float)
ball_pred_all = np.zeros_like(y, dtype=float)
hbp_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(32, activation = 'relu'),
        Dense(3, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    strike_pred_all[test_idx] = preds[:, 0]
    ball_pred_all[test_idx] = preds[:, 1]
    hbp_pred_all[test_idx] = preds[:, 2]


df_take = df_take.with_columns(pl.Series("called_strike_pred_nn", strike_pred_all),
                               pl.Series("called_ball_pred_nn", ball_pred_all),
                               pl.Series("called_hbp_pred_nn", hbp_pred_all))

In [ ]:
calibration_plot(data = df_take, preds = 'called_strike_pred_nn', actuals = 'called_strike', model = 'Neural Net 2 Layers Called Model (Strikes)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_ball_pred_nn', actuals = 'called_ball', model = 'Neural Net 2 Layers Called Model (Balls)')

In [ ]:
calibration_plot(data = df_take, preds = 'called_hbp_pred_nn', actuals = 'called_hbp', model = 'Neural Net 2 Layers Called Model (HBPs)')

# Contact/Whiff Model (XGBoost Sigmoid)

In [ ]:
# filter to relevant rows
df_contact = df.filter(pl.col('swing') == 1)

In [ ]:
df_contact['contact'].mean()

## XGBoost

In [ ]:
# xgboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'contact'

df_pandas = df_contact.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBClassifier()
preds = cross_val_predict(model, X, y , cv=5, method='predict_proba')

df_contact = df_contact.with_columns(pl.Series("whiff_pred_xgb", preds[:, 0]),
                               pl.Series("contact_pred_xgb", preds[:, 1]))

In [ ]:
calibration_plot(data = df_contact, preds = 'contact_pred_xgb', actuals = 'contact', model = 'XGBoost Contact Model')

## XGboost Isotonic

In [ ]:
# xgboost isotonic model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'contact'

df_pandas = df_contact.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

contact_pred_all = np.zeros_like(y, dtype=float)
whiff_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier()

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'isotonic', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    contact_pred_all[test_idx] = preds[:, 1]
    whiff_pred_all[test_idx] = preds[:, 0]


df_contact = df_contact.with_columns(pl.Series("contact_pred_xgb_calibrated", contact_pred_all),
                               pl.Series("whiff_pred_xgb_calibrated", whiff_pred_all))

In [ ]:
calibration_plot(data = df_contact, preds = 'contact_pred_xgb_calibrated', actuals = 'contact', model = 'XGBoost Contact Model Calibrated')

## XGBoost Sigmoid

In [ ]:
# xgboost sigmoid model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'contact'

df_pandas = df_contact.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

contact_pred_all = np.zeros_like(y, dtype=float)
whiff_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier()

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'sigmoid', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    contact_pred_all[test_idx] = preds[:, 1]
    whiff_pred_all[test_idx] = preds[:, 0]


df_contact = df_contact.with_columns(pl.Series("contact_pred_xgb_calibrated", contact_pred_all),
                               pl.Series("whiff_pred_xgb_calibrated", whiff_pred_all))

In [ ]:
calibration_plot(data = df_contact, preds = 'contact_pred_xgb_calibrated', actuals = 'contact', model = 'XGBoost Contact Model Calibrated (Sigmoid)')

## Neural Net 4 Layers

In [ ]:
# Neural net 4 layers
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'contact'

df_pandas = df_contact.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

y_pred_all = np.zeros_like(y, dtype=float)
whiff_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1], )),
        Dense(128, activation = 'relu'),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ])
    model.compile(optimizer = Adam(), loss = 'binary_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, batch_size = 1024, epochs = 15, callbacks = [early_stop], verbose = 1)

    y_pred = model.predict(X_test_scaled).flatten()

    y_pred_all[test_idx] = y_pred

df_contact = df_contact.with_columns(pl.Series("contact_pred_nn", y_pred_all),
                                     pl.Series("whiff_pred_nn", 1 - y_pred_all))

In [ ]:
calibration_plot(data = df_contact, preds = 'contact_pred_nn', actuals = 'contact', model = 'Neural Net 4 Layers Contact Model')

## Neural Net 3 Layers

In [ ]:
# Neural net 3 layers
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'contact'

df_pandas = df_contact.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

y_pred_all = np.zeros_like(y, dtype=float)
whiff_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1], )),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ])
    model.compile(optimizer = Adam(), loss = 'binary_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, batch_size = 1024, epochs = 15, callbacks = [early_stop], verbose = 1)

    y_pred = model.predict(X_test_scaled).flatten()

    y_pred_all[test_idx] = y_pred

df_contact = df_contact.with_columns(pl.Series("contact_pred_nn", y_pred_all),
                                     pl.Series("whiff_pred_nn", 1 - y_pred_all))

In [ ]:
calibration_plot(data = df_contact, preds = 'contact_pred_nn', actuals = 'contact', model = 'Neural Net 3 Layers Contact Model')

# Batted Ball Model (3 Layer DNN)

In [ ]:
# filter to relevant rows
df_bb = df.filter(pl.col('contact') == 1)

# create target col
df_bb = df_bb.with_columns(
    pl.when(pl.col('bb_type') == 'foul')
    .then(0)
    .when(pl.col('bb_type') == 'ground_ball')
    .then(1)
    .when(pl.col('bb_type') == 'line_drive')
    .then(2)
    .when(pl.col('bb_type') == 'fly_ball')
    .then(3)
    .when(pl.col('bb_type') == 'popup')
    .then(4)
    .alias('bb_target')
)

In [ ]:
df_bb['foul'].mean()

In [ ]:
df_bb['ground_ball'].mean()

In [ ]:
df_bb['line_drive'].mean()

In [ ]:
df_bb['fly_ball'].mean()

In [ ]:
df_bb['popup'].mean()

## XGBoost

In [ ]:
# xgboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'bb_target'

df_pandas = df_bb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBClassifier(objective = 'multi:softprob', num_class = 5, eval_metric = 'mlogloss')
preds = cross_val_predict(model, X, y , cv=5, method='predict_proba')

df_bb = df_bb.with_columns(pl.Series("foul_pred_xgb", preds[:, 0]),
                               pl.Series("gb_pred_xgb", preds[:, 1]),
                               pl.Series("ld_pred_xgb", preds[:, 2]),
                               pl.Series("fb_pred_xgb", preds[:, 3]),
                               pl.Series("po_pred_xgb", preds[:, 4]))

In [ ]:
calibration_plot(data = df_bb, preds = 'foul_pred_xgb', actuals = 'foul', model = 'XGBoost Batted Ball Model (Foul)')

In [ ]:
calibration_plot(data = df_bb, preds = 'gb_pred_xgb', actuals = 'ground_ball', model = 'XGBoost Batted Ball Model (Ground Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'ld_pred_xgb', actuals = 'line_drive', model = 'XGBoost Batted Ball Model (Line Drive)')

In [ ]:
calibration_plot(data = df_bb, preds = 'fb_pred_xgb', actuals = 'fly_ball', model = 'XGBoost Batted Ball Model (Fly Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'po_pred_xgb', actuals = 'popup', model = 'XGBoost Batted Ball Model (Popup)')

## XGBoost Isotonic

In [ ]:
# xgboost isotonic model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'bb_target'

df_pandas = df_bb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

foul_pred_all = np.zeros_like(y, dtype=float)
gb_pred_all = np.zeros_like(y, dtype=float)
ld_pred_all = np.zeros_like(y, dtype=float)
fb_pred_all = np.zeros_like(y, dtype=float)
po_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier(objective = 'multi:softprob', num_class = 5, eval_metric = 'mlogloss')

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'isotonic', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    foul_pred_all[test_idx] = preds[:, 0]
    gb_pred_all[test_idx] = preds[:, 1]
    ld_pred_all[test_idx] = preds[:, 2]
    fb_pred_all[test_idx] = preds[:, 3]
    po_pred_all[test_idx] = preds[:, 4]

df_bb = df_bb.with_columns(pl.Series("foul_pred_xgb", foul_pred_all),
                               pl.Series("gb_pred_xgb", gb_pred_all),
                               pl.Series("ld_pred_xgb", ld_pred_all),
                               pl.Series("fb_pred_xgb", fb_pred_all),
                               pl.Series("po_pred_xgb", po_pred_all))

In [ ]:
calibration_plot(data = df_bb, preds = 'foul_pred_xgb', actuals = 'foul', model = 'XGBoost Batted Ball Model Calibrated Isotonic (Foul)')

In [ ]:
calibration_plot(data = df_bb, preds = 'gb_pred_xgb', actuals = 'ground_ball', model = 'XGBoost Batted Ball Model Calibrated Isotonic (Ground Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'ld_pred_xgb', actuals = 'line_drive', model = 'XGBoost Batted Ball Model Calibrated Isotonic (Line Drive)')

In [ ]:
calibration_plot(data = df_bb, preds = 'fb_pred_xgb', actuals = 'fly_ball', model = 'XGBoost Batted Ball Model Calibrated Isotonic (Fly Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'po_pred_xgb', actuals = 'popup', model = 'XGBoost Batted Ball Model Calibrated Isotonic (Pop Up)')

## XGBoost Sigmoid

In [ ]:
# xgboost sigmoid model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'bb_target'

df_pandas = df_bb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

foul_pred_all = np.zeros_like(y, dtype=float)
gb_pred_all = np.zeros_like(y, dtype=float)
ld_pred_all = np.zeros_like(y, dtype=float)
fb_pred_all = np.zeros_like(y, dtype=float)
po_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier(objective = 'multi:softprob', num_class = 5, eval_metric = 'mlogloss')

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'sigmoid', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    foul_pred_all[test_idx] = preds[:, 0]
    gb_pred_all[test_idx] = preds[:, 1]
    ld_pred_all[test_idx] = preds[:, 2]
    fb_pred_all[test_idx] = preds[:, 3]
    po_pred_all[test_idx] = preds[:, 4]

df_bb = df_bb.with_columns(pl.Series("foul_pred_xgb", foul_pred_all),
                               pl.Series("gb_pred_xgb", gb_pred_all),
                               pl.Series("ld_pred_xgb", ld_pred_all),
                               pl.Series("fb_pred_xgb", fb_pred_all),
                               pl.Series("po_pred_xgb", po_pred_all))

In [ ]:
calibration_plot(data = df_bb, preds = 'foul_pred_xgb', actuals = 'foul', model = 'XGBoost Batted Ball Model Calibrated Sigmoid (Foul)')

In [ ]:
calibration_plot(data = df_bb, preds = 'gb_pred_xgb', actuals = 'ground_ball', model = 'XGBoost Batted Ball Model Calibrated Sigmoid (Ground Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'ld_pred_xgb', actuals = 'line_drive', model = 'XGBoost Batted Ball Model Calibrated Sigmoid (Line Drive)')

In [ ]:
calibration_plot(data = df_bb, preds = 'fb_pred_xgb', actuals = 'fly_ball', model = 'XGBoost Batted Ball Model Calibrated Sigmoid (Fly Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'po_pred_xgb', actuals = 'popup', model = 'XGBoost Batted Ball Model Calibrated Sigmoid (Pop Up)')

## Neural Net 4 Layers

In [ ]:
# Neural net 4 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'bb_target'

df_pandas = df_bb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

foul_pred_all = np.zeros_like(y, dtype=float)
gb_pred_all = np.zeros_like(y, dtype=float)
ld_pred_all = np.zeros_like(y, dtype=float)
fb_pred_all = np.zeros_like(y, dtype=float)
po_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(128, activation = 'relu'),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    foul_pred_all[test_idx] = preds[:, 0]
    gb_pred_all[test_idx] = preds[:, 1]
    ld_pred_all[test_idx] = preds[:, 2]
    fb_pred_all[test_idx] = preds[:, 3]
    po_pred_all[test_idx] = preds[:, 4]


df_bb = df_bb.with_columns(pl.Series("foul_pred_nn", foul_pred_all),
                               pl.Series("gb_pred_nn", gb_pred_all),
                               pl.Series("ld_pred_nn", ld_pred_all),
                               pl.Series("fb_pred_nn", fb_pred_all),
                               pl.Series("po_pred_nn", po_pred_all))

In [ ]:
calibration_plot(data = df_bb, preds = 'foul_pred_nn', actuals = 'foul', model = 'Neural Net 4 Layers Batted Ball Model (Foul)')

In [ ]:
calibration_plot(data = df_bb, preds = 'gb_pred_nn', actuals = 'ground_ball', model = 'Neural Net 4 Layers Batted Ball Model (Ground Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'ld_pred_nn', actuals = 'line_drive', model = 'Neural Net 4 Layers Batted Ball Model (Line Drive)')

In [ ]:
calibration_plot(data = df_bb, preds = 'fb_pred_nn', actuals = 'fly_ball', model = 'Neural Net 4 Layers Batted Ball Model (Fly Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'po_pred_nn', actuals = 'popup', model = 'Neural Net 4 Layers Batted Ball Model (Pop Up)')

## Neural Net 3 Layers

In [ ]:
# Neural net 3 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'bb_target'

df_pandas = df_bb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

foul_pred_all = np.zeros_like(y, dtype=float)
gb_pred_all = np.zeros_like(y, dtype=float)
ld_pred_all = np.zeros_like(y, dtype=float)
fb_pred_all = np.zeros_like(y, dtype=float)
po_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    foul_pred_all[test_idx] = preds[:, 0]
    gb_pred_all[test_idx] = preds[:, 1]
    ld_pred_all[test_idx] = preds[:, 2]
    fb_pred_all[test_idx] = preds[:, 3]
    po_pred_all[test_idx] = preds[:, 4]


df_bb = df_bb.with_columns(pl.Series("foul_pred_nn", foul_pred_all),
                               pl.Series("gb_pred_nn", gb_pred_all),
                               pl.Series("ld_pred_nn", ld_pred_all),
                               pl.Series("fb_pred_nn", fb_pred_all),
                               pl.Series("po_pred_nn", po_pred_all))

In [ ]:
calibration_plot(data = df_bb, preds = 'foul_pred_nn', actuals = 'foul', model = 'Neural Net 3 Layers Batted Ball Model (Foul)')

In [ ]:
calibration_plot(data = df_bb, preds = 'gb_pred_nn', actuals = 'ground_ball', model = 'Neural Net 3 Layers Batted Ball Model (Ground Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'ld_pred_nn', actuals = 'line_drive', model = 'Neural Net 3 Layers Batted Ball Model (Line Drive)')

In [ ]:
calibration_plot(data = df_bb, preds = 'fb_pred_nn', actuals = 'fly_ball', model = 'Neural Net 3 Layers Batted Ball Model (Fly Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'po_pred_nn', actuals = 'popup', model = 'Neural Net 3 LayersBatted Ball Model (Pop Up)')

## Neural Net 2 Layers

In [ ]:
# Neural net 2 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'bb_target'

df_pandas = df_bb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

foul_pred_all = np.zeros_like(y, dtype=float)
gb_pred_all = np.zeros_like(y, dtype=float)
ld_pred_all = np.zeros_like(y, dtype=float)
fb_pred_all = np.zeros_like(y, dtype=float)
po_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    foul_pred_all[test_idx] = preds[:, 0]
    gb_pred_all[test_idx] = preds[:, 1]
    ld_pred_all[test_idx] = preds[:, 2]
    fb_pred_all[test_idx] = preds[:, 3]
    po_pred_all[test_idx] = preds[:, 4]


df_bb = df_bb.with_columns(pl.Series("foul_pred_nn", foul_pred_all),
                               pl.Series("gb_pred_nn", gb_pred_all),
                               pl.Series("ld_pred_nn", ld_pred_all),
                               pl.Series("fb_pred_nn", fb_pred_all),
                               pl.Series("po_pred_nn", po_pred_all))

In [ ]:
calibration_plot(data = df_bb, preds = 'foul_pred_nn', actuals = 'foul', model = 'Neural Net 2 Layers Batted Ball Model (Foul)')

In [ ]:
calibration_plot(data = df_bb, preds = 'gb_pred_nn', actuals = 'ground_ball', model = 'Neural Net 2 Layers Batted Ball Model (Ground Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'ld_pred_nn', actuals = 'line_drive', model = 'Neural Net 2 Layers Batted Ball Model (Line Drive)')

In [ ]:
calibration_plot(data = df_bb, preds = 'fb_pred_nn', actuals = 'fly_ball', model = 'Neural Net 2 Layers Batted Ball Model (Fly Ball)')

In [ ]:
calibration_plot(data = df_bb, preds = 'po_pred_nn', actuals = 'popup', model = 'Neural Net 2 Layers Batted Ball Model (Pop Up)')

# Groundball Model (4 Layer DNN)

In [ ]:
# filter to relevant rows
df_gb = df.filter(pl.col('bb_type') == 'ground_ball')

# create target col
df_gb = df_gb.with_columns(
    pl.when(pl.col('out') == 1)
    .then(0)
    .when(pl.col('single') == 1)
    .then(1)
    .when(pl.col('double') == 1)
    .then(2)
    .when(pl.col('triple') == 1)
    .then(3)
    .alias('gb_target')
)

In [ ]:
df_gb['out'].mean()

In [ ]:
df_gb['single'].mean()

In [ ]:
df_gb['double'].mean()

In [ ]:
df_gb['triple'].mean()

## XGBoost

In [ ]:
# xgboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'gb_target'

df_pandas = df_gb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBClassifier(objective = 'multi:softprob', num_class = 4, eval_metric = 'mlogloss')
preds = cross_val_predict(model, X, y , cv=5, method='predict_proba')

df_gb = df_gb.with_columns(pl.Series("out_pred_xgb", preds[:, 0]),
                               pl.Series("single_pred_xgb", preds[:, 1]),
                               pl.Series("double_pred_xgb", preds[:, 2]),
                               pl.Series("triple_pred_xgb", preds[:, 3]))

In [ ]:
calibration_plot(data = df_gb, preds = 'out_pred_xgb', actuals = 'out', model = 'XGBoost Ground Ball Model (Outs)')

In [ ]:
calibration_plot(data = df_gb, preds = 'single_pred_xgb', actuals = 'single', model = 'XGBoost Ground Ball Model (Singles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'double_pred_xgb', actuals = 'double', model = 'XGBoost Ground Ball Model (Doubles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'triple_pred_xgb', actuals = 'triple', model = 'XGBoost Ground Ball Model (Triples)')

## XGBoost Isotonic

In [ ]:
# xgboost isotonic model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'gb_target'

df_pandas = df_gb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier(objective = 'multi:softprob', num_class = 4, eval_metric = 'mlogloss')

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'isotonic', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]

df_gb = df_gb.with_columns(pl.Series("out_pred_xgb", out_pred_all),
                               pl.Series("single_pred_xgb", single_pred_all),
                               pl.Series("double_pred_xgb", double_pred_all),
                               pl.Series("triple_pred_xgb", triple_pred_all))

In [ ]:
calibration_plot(data = df_gb, preds = 'out_pred_xgb', actuals = 'out', model = 'XGBoost Ground Ball Model Isotonic Calibrated (Outs)')

In [ ]:
calibration_plot(data = df_gb, preds = 'single_pred_xgb', actuals = 'single', model = 'XGBoost Ground Ball Model Isotonic Calibrated (Singles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'double_pred_xgb', actuals = 'double', model = 'XGBoost Ground Ball Model Isotonic Calibrated (Doubles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'triple_pred_xgb', actuals = 'triple', model = 'XGBoost Ground Ball Model Isotonic Calibrated (Triples)')

## XGBoost Sigmoid

In [ ]:
# xgboost sigmoid model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'gb_target'

df_pandas = df_gb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier(objective = 'multi:softprob', num_class = 4, eval_metric = 'mlogloss')

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'sigmoid', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]

df_gb = df_gb.with_columns(pl.Series("out_pred_xgb", out_pred_all),
                               pl.Series("single_pred_xgb", single_pred_all),
                               pl.Series("double_pred_xgb", double_pred_all),
                               pl.Series("triple_pred_xgb", triple_pred_all))

In [ ]:
calibration_plot(data = df_gb, preds = 'out_pred_xgb', actuals = 'out', model = 'XGBoost Ground Ball Model Sigmoid Calibrated (Outs)')

In [ ]:
calibration_plot(data = df_gb, preds = 'single_pred_xgb', actuals = 'single', model = 'XGBoost Ground Ball Model Sigmoid Calibrated (Singles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'double_pred_xgb', actuals = 'double', model = 'XGBoost Ground Ball Model Sigmoid Calibrated (Doubles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'triple_pred_xgb', actuals = 'triple', model = 'XGBoost Ground Ball Model Sigmoid Calibrated (Triples)')

## Neural Net 4 Layers

In [ ]:
# Neural net 4 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'gb_target'

df_pandas = df_gb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(128, activation = 'relu'),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(4, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]


df_gb = df_gb.with_columns(pl.Series("out_pred_nn", out_pred_all),
                               pl.Series("single_pred_nn", single_pred_all),
                               pl.Series("double_pred_nn", double_pred_all),
                               pl.Series("triple_pred_nn", triple_pred_all))

In [ ]:
calibration_plot(data = df_gb, preds = 'out_pred_nn', actuals = 'out', model = 'Neural Net 4 Layers Ground Ball Model (Outs)')

In [ ]:
calibration_plot(data = df_gb, preds = 'single_pred_nn', actuals = 'single', model = 'Neural Net 4 Layers Ground Ball Model (Singles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'double_pred_nn', actuals = 'double', model = 'Neural Net 4 Layers Ground Ball Model (Doubles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'triple_pred_nn', actuals = 'triple', model = 'Neural Net 4 Layers Ground Ball Model (Triples)')

## Neural Net 3 Layers

In [ ]:
# Neural net 3 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'gb_target'

df_pandas = df_gb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(4, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]


df_gb = df_gb.with_columns(pl.Series("out_pred_nn", out_pred_all),
                               pl.Series("single_pred_nn", single_pred_all),
                               pl.Series("double_pred_nn", double_pred_all),
                               pl.Series("triple_pred_nn", triple_pred_all))

In [ ]:
calibration_plot(data = df_gb, preds = 'out_pred_nn', actuals = 'out', model = 'Neural Net 3 Layers Ground Ball Model (Outs)')

In [ ]:
calibration_plot(data = df_gb, preds = 'single_pred_nn', actuals = 'single', model = 'Neural Net 3 Layers Ground Ball Model (Singles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'double_pred_nn', actuals = 'double', model = 'Neural Net 3 Layers Ground Ball Model (Doubles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'triple_pred_nn', actuals = 'triple', model = 'Neural Net 3 Layers Ground Ball Model (Triples)')

## Neural Net 2 Layers

In [ ]:
# Neural net 2 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'gb_target'

df_pandas = df_gb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(32, activation = 'relu'),
        Dense(4, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]


df_gb = df_gb.with_columns(pl.Series("out_pred_nn", out_pred_all),
                               pl.Series("single_pred_nn", single_pred_all),
                               pl.Series("double_pred_nn", double_pred_all),
                               pl.Series("triple_pred_nn", triple_pred_all))

In [ ]:
calibration_plot(data = df_gb, preds = 'out_pred_nn', actuals = 'out', model = 'Neural Net 2 Layers Ground Ball Model (Outs)')

In [ ]:
calibration_plot(data = df_gb, preds = 'single_pred_nn', actuals = 'single', model = 'Neural Net 2 Layers Ground Ball Model (Singles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'double_pred_nn', actuals = 'double', model = 'Neural Net 2 Layers Ground Ball Model (Doubles)')

In [ ]:
calibration_plot(data = df_gb, preds = 'triple_pred_nn', actuals = 'triple', model = 'Neural Net 2 Layers Ground Ball Model (Triples)')

# Line Drive Model (2 Layer DNN)

In [ ]:
# filter to relevant rows
df_ld = df.filter(pl.col('bb_type') == 'line_drive')

# create target col
df_ld = df_ld.with_columns(
    pl.when(pl.col('out') == 1)
    .then(0)
    .when(pl.col('single') == 1)
    .then(1)
    .when(pl.col('double') == 1)
    .then(2)
    .when(pl.col('triple') == 1)
    .then(3)
    .when(pl.col('home_run') == 1)
    .then(4)
    .alias('ld_target')
)

In [ ]:
df_ld['out'].mean()

In [ ]:
df_ld['single'].mean()

In [ ]:
df_ld['double'].mean()

In [ ]:
df_ld['triple'].mean()

In [ ]:
df_ld['home_run'].mean()

## XGBoost

In [ ]:
# xgboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'ld_target'

df_pandas = df_ld.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBClassifier(objective = 'multi:softprob', num_class = 5, eval_metric = 'mlogloss')
preds = cross_val_predict(model, X, y , cv=5, method='predict_proba')

df_ld = df_ld.with_columns(pl.Series("out_pred_xgb", preds[:, 0]),
                            pl.Series("single_pred_xgb", preds[:, 1]),
                            pl.Series("double_pred_xgb", preds[:, 2]),
                            pl.Series("triple_pred_xgb", preds[:, 3]),
                            pl.Series("homerun_pred_xgb", preds[:, 4]))

In [ ]:
calibration_plot(data = df_ld, preds = 'out_pred_xgb', actuals = 'out', model = 'XGBoost Line Drive Model (Outs)')

In [ ]:
calibration_plot(data = df_ld, preds = 'single_pred_xgb', actuals = 'single', model = 'XGBoost Line Drive Model (Singles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'double_pred_xgb', actuals = 'double', model = 'XGBoost Line Drive Model (Doubles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'triple_pred_xgb', actuals = 'triple', model = 'XGBoost Line Drive Model (Triples)')

In [ ]:
calibration_plot(data = df_ld, preds = 'homerun_pred_xgb', actuals = 'home_run', model = 'XGBoost Line Drive Model (Home Runs)')

## XGBoost Isotonic

In [ ]:
# xgboost isotonic model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'ld_target'

df_pandas = df_ld.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
homerun_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier(objective = 'multi:softprob', num_class = 5, eval_metric = 'mlogloss')

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'isotonic', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]

df_ld = df_ld.with_columns(pl.Series("out_pred_xgb", out_pred_all),
                               pl.Series("single_pred_xgb", single_pred_all),
                               pl.Series("double_pred_xgb", double_pred_all),
                               pl.Series("triple_pred_xgb", triple_pred_all),
                           pl.Series("homerun_pred_xgb", homerun_pred_all))

In [ ]:
calibration_plot(data = df_ld, preds = 'out_pred_xgb', actuals = 'out', model = 'XGBoost Line Drive Model Isotonic Calibrated (Outs)')

In [ ]:
calibration_plot(data = df_ld, preds = 'single_pred_xgb', actuals = 'single', model = 'XGBoost Line Drive Model Isotonic Calibrated (Singles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'double_pred_xgb', actuals = 'double', model = 'XGBoost Line Drive Model Isotonic Calibrated (Doubles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'triple_pred_xgb', actuals = 'triple', model = 'XGBoost Line Drive Model Isotonic Calibrated (Triples)')

In [ ]:
calibration_plot(data = df_ld, preds = 'homerun_pred_xgb', actuals = 'home_run', model = 'XGBoost Line Drive Model Isotonic Calibrated (Home Runs)')

## XGBoost Sigmoid

In [ ]:
# xgboost sigmoid model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'ld_target'

df_pandas = df_ld.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
homerun_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier(objective = 'multi:softprob', num_class = 5, eval_metric = 'mlogloss')

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'sigmoid', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]

df_ld = df_ld.with_columns(pl.Series("out_pred_xgb", out_pred_all),
                               pl.Series("single_pred_xgb", single_pred_all),
                               pl.Series("double_pred_xgb", double_pred_all),
                               pl.Series("triple_pred_xgb", triple_pred_all),
                           pl.Series("homerun_pred_xgb", homerun_pred_all))

In [ ]:
calibration_plot(data = df_ld, preds = 'out_pred_xgb', actuals = 'out', model = 'XGBoost Line Drive Model Sigmoid Calibrated (Outs)')

In [ ]:
calibration_plot(data = df_ld, preds = 'single_pred_xgb', actuals = 'single', model = 'XGBoost Line Drive Model Sigmoid Calibrated (Singles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'double_pred_xgb', actuals = 'double', model = 'XGBoost Line Drive Model Sigmoid Calibrated (Doubles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'triple_pred_xgb', actuals = 'triple', model = 'XGBoost Line Drive Model Sigmoid Calibrated (Triples)')

In [ ]:
calibration_plot(data = df_ld, preds = 'homerun_pred_xgb', actuals = 'home_run', model = 'XGBoost Line Drive Model Sigmoid Calibrated (Home Runs)')

## Neural Net 2 Layers

In [ ]:
# Neural net 2 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'ld_target'

df_pandas = df_ld.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
homerun_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]


df_ld = df_ld.with_columns(pl.Series("out_pred_nn", out_pred_all),
                               pl.Series("single_pred_nn", single_pred_all),
                               pl.Series("double_pred_nn", double_pred_all),
                               pl.Series("triple_pred_nn", triple_pred_all),
                           pl.Series("homerun_pred_nn", homerun_pred_all))

In [ ]:
calibration_plot(data = df_ld, preds = 'out_pred_nn', actuals = 'out', model = 'Neural Net 2 Layers Line Drive Model (Outs)')

In [ ]:
calibration_plot(data = df_ld, preds = 'single_pred_nn', actuals = 'single', model = 'Neural Net 2 Layers Line Drive Model (Singles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'double_pred_nn', actuals = 'double', model = 'Neural Net 2 Layers Line Drive Model (Doubles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'triple_pred_nn', actuals = 'triple', model = 'Neural Net 2 Layers Line Drive Model (Triples)')

In [ ]:
calibration_plot(data = df_ld, preds = 'homerun_pred_nn', actuals = 'home_run', model = 'Neural Net 2 Layers Line Drive Model (Home Runs)')

## Neural Net 3 Layers

In [ ]:
# Neural net 3 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'ld_target'

df_pandas = df_ld.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
homerun_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]


df_ld = df_ld.with_columns(pl.Series("out_pred_nn", out_pred_all),
                               pl.Series("single_pred_nn", single_pred_all),
                               pl.Series("double_pred_nn", double_pred_all),
                               pl.Series("triple_pred_nn", triple_pred_all),
                           pl.Series("homerun_pred_nn", homerun_pred_all))

In [ ]:
calibration_plot(data = df_ld, preds = 'out_pred_nn', actuals = 'out', model = 'Neural Net 3 Layers Line Drive Model (Outs)')

In [ ]:
calibration_plot(data = df_ld, preds = 'single_pred_nn', actuals = 'single', model = 'Neural Net 3 Layers Line Drive Model (Singles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'double_pred_nn', actuals = 'double', model = 'Neural Net 3 Layers Line Drive Model (Doubles)')

In [ ]:
calibration_plot(data = df_ld, preds = 'triple_pred_nn', actuals = 'triple', model = 'Neural Net 3 Layers Line Drive Model (Triples)')

In [ ]:
calibration_plot(data = df_ld, preds = 'homerun_pred_nn', actuals = 'home_run', model = 'Neural Net 3 Layers Line Drive Model (Home Runs)')

# Flyball Model (2 Layer DNN)

In [ ]:
# filter to relevant rows
df_fb = df.filter(pl.col('bb_type') == 'fly_ball')

# create target col
df_fb = df_fb.with_columns(
    pl.when(pl.col('out') == 1)
    .then(0)
    .when(pl.col('single') == 1)
    .then(1)
    .when(pl.col('double') == 1)
    .then(2)
    .when(pl.col('triple') == 1)
    .then(3)
    .when(pl.col('home_run') == 1)
    .then(4)
    .alias('fb_target')
)

In [ ]:
df_fb['out'].mean()

In [ ]:
df_fb['single'].mean()

In [ ]:
df_fb['double'].mean()

In [ ]:
df_fb['triple'].mean()

In [ ]:
df_fb['home_run'].mean()

## XGBoost

In [ ]:
# xgboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'fb_target'

df_pandas = df_fb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBClassifier(objective = 'multi:softprob', num_class = 5, eval_metric = 'mlogloss')
preds = cross_val_predict(model, X, y , cv=5, method='predict_proba')

df_fb = df_fb.with_columns(pl.Series("out_pred_xgb", preds[:, 0]),
                            pl.Series("single_pred_xgb", preds[:, 1]),
                            pl.Series("double_pred_xgb", preds[:, 2]),
                            pl.Series("triple_pred_xgb", preds[:, 3]),
                            pl.Series("homerun_pred_xgb", preds[:, 4]))

In [ ]:
calibration_plot(data = df_fb, preds = 'out_pred_xgb', actuals = 'out', model = 'XGBoost Fly Ball Model (Outs)')

In [ ]:
calibration_plot(data = df_fb, preds = 'single_pred_xgb', actuals = 'single', model = 'XGBoost Fly Ball Model (Singles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'double_pred_xgb', actuals = 'double', model = 'XGBoost Fly Ball Model (Doubles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'triple_pred_xgb', actuals = 'triple', model = 'XGBoost Fly Ball Model (Triples)')

In [ ]:
calibration_plot(data = df_fb, preds = 'homerun_pred_xgb', actuals = 'home_run', model = 'XGBoost Fly Ball Model (Home Runs)')

## XGBoost Isotonic

In [ ]:
# xgboost isotonic model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'fb_target'

df_pandas = df_fb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
homerun_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier(objective = 'multi:softprob', num_class = 5, eval_metric = 'mlogloss')

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'isotonic', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]

df_fb = df_fb.with_columns(pl.Series("out_pred_xgb", out_pred_all),
                               pl.Series("single_pred_xgb", single_pred_all),
                               pl.Series("double_pred_xgb", double_pred_all),
                               pl.Series("triple_pred_xgb", triple_pred_all),
                           pl.Series("homerun_pred_xgb", homerun_pred_all))

In [ ]:
calibration_plot(data = df_fb, preds = 'out_pred_xgb', actuals = 'out', model = 'XGBoost Fly Ball Model Isotonic Calibrated (Outs)')

In [ ]:
calibration_plot(data = df_fb, preds = 'single_pred_xgb', actuals = 'single', model = 'XGBoost Fly Ball Model Isotonic Calibrated (Singles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'double_pred_xgb', actuals = 'double', model = 'XGBoost Fly Ball Model Isotonic Calibrated (Doubles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'triple_pred_xgb', actuals = 'triple', model = 'XGBoost Fly Ball Model Isotonic Calibrated (Triples)')

In [ ]:
calibration_plot(data = df_fb, preds = 'homerun_pred_xgb', actuals = 'home_run', model = 'XGBoost Fly Ball Model Isotonic Calibrated (Home Runs)')

## XGBoost Sigmoid

In [ ]:
# xgboost sigmoid model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'fb_target'

df_pandas = df_fb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
homerun_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    xgb = XGBClassifier(objective = 'multi:softprob', num_class = 5, eval_metric = 'mlogloss')

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'sigmoid', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    preds = calibrated_xgb.predict_proba(X_test)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]

df_fb = df_fb.with_columns(pl.Series("out_pred_xgb", out_pred_all),
                               pl.Series("single_pred_xgb", single_pred_all),
                               pl.Series("double_pred_xgb", double_pred_all),
                               pl.Series("triple_pred_xgb", triple_pred_all),
                           pl.Series("homerun_pred_xgb", homerun_pred_all))

In [ ]:
calibration_plot(data = df_fb, preds = 'out_pred_xgb', actuals = 'out', model = 'XGBoost Fly Ball Model Sigmoid Calibrated (Outs)')

In [ ]:
calibration_plot(data = df_fb, preds = 'single_pred_xgb', actuals = 'single', model = 'XGBoost Fly Ball Model Sigmoid Calibrated (Singles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'double_pred_xgb', actuals = 'double', model = 'XGBoost Fly Ball Model Sigmoid Calibrated (Doubles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'triple_pred_xgb', actuals = 'triple', model = 'XGBoost Fly Ball Model Sigmoid Calibrated (Triples)')

In [ ]:
calibration_plot(data = df_fb, preds = 'homerun_pred_xgb', actuals = 'home_run', model = 'XGBoost Fly Ball Model Sigmoid Calibrated (Home Runs)')

## Neural Net 2 Layers

In [ ]:
# Neural net 2 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'fb_target'

df_pandas = df_fb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
homerun_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 1,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 64, verbose = 1)

    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]


df_fb = df_fb.with_columns(pl.Series("out_pred_nn", out_pred_all),
                               pl.Series("single_pred_nn", single_pred_all),
                               pl.Series("double_pred_nn", double_pred_all),
                               pl.Series("triple_pred_nn", triple_pred_all),
                           pl.Series("homerun_pred_nn", homerun_pred_all))

In [ ]:
calibration_plot(data = df_fb, preds = 'out_pred_nn', actuals = 'out', model = 'Neural Net 2 Layers Fly Ball Model (Outs)')

In [ ]:
calibration_plot(data = df_fb, preds = 'single_pred_nn', actuals = 'single', model = 'Neural Net 2 Layers Fly Ball Model (Singles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'double_pred_nn', actuals = 'double', model = 'Neural Net 2 Layers Fly Ball Model (Doubles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'triple_pred_nn', actuals = 'triple', model = 'Neural Net 2 Layers Fly Ball Model (Triples)')

In [ ]:
calibration_plot(data = df_fb, preds = 'homerun_pred_nn', actuals = 'home_run', model = 'Neural Net 2 Layers Fly Ball Model (Home Runs)')

## Neural Net 4 Layers

In [ ]:
# Neural net 4 layer model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'fb_target'

df_pandas = df_fb.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros_like(y, dtype=float)
single_pred_all = np.zeros_like(y, dtype=float)
double_pred_all = np.zeros_like(y, dtype=float)
triple_pred_all = np.zeros_like(y, dtype=float)
homerun_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1],)),
        Dense(128, activation = 'relu'),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 1,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 64, verbose = 1)

    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]


df_fb = df_fb.with_columns(pl.Series("out_pred_nn", out_pred_all),
                               pl.Series("single_pred_nn", single_pred_all),
                               pl.Series("double_pred_nn", double_pred_all),
                               pl.Series("triple_pred_nn", triple_pred_all),
                           pl.Series("homerun_pred_nn", homerun_pred_all))

In [ ]:
calibration_plot(data = df_fb, preds = 'out_pred_nn', actuals = 'out', model = 'Neural Net 4 Layers Fly Ball Model (Outs)')

In [ ]:
calibration_plot(data = df_fb, preds = 'single_pred_nn', actuals = 'single', model = 'Neural Net 4 Layers Fly Ball Model (Singles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'double_pred_nn', actuals = 'double', model = 'Neural Net 4 Layers Fly Ball Model (Doubles)')

In [ ]:
calibration_plot(data = df_fb, preds = 'triple_pred_nn', actuals = 'triple', model = 'Neural Net 4 Layers Fly Ball Model (Triples)')

In [ ]:
calibration_plot(data = df_fb, preds = 'homerun_pred_nn', actuals = 'home_run', model = 'Neural Net 4 Layers Fly Ball Model (Home Runs)')

# First Model

## Swing Model

In [ ]:
# xgboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'swing'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBClassifier()
preds = cross_val_predict(model, X, y , cv=5, method='predict_proba')

df = df.with_columns(pl.Series("swing_pred_xgb", preds[:, 1]),
                     pl.Series("take_pred_xgb", preds[:, 0]))

In [ ]:
df.write_csv('df_swings.csv')
files.download('df_swings.csv')

## Strike/Ball/HBP Model

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_swings.csv')

df = df.with_columns([
    pl.when(pl.col('called_strike') == 1)
    .then(0)
    .when(pl.col('called_ball') ==  1)
    .then(1)
    .otherwise(2)
    .alias('take_target')
])

In [ ]:
# Neural net 3 dense layer model
feats = ['swing'] + ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed',
                     'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = ['take_target']

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

strike_pred_all = np.zeros(y.shape[0], dtype=float)
ball_pred_all = np.zeros(y.shape[0], dtype=float)
hbp_pred_all = np.zeros(y.shape[0], dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")
    # Split into train/test datasets
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    # Filter train dataset to only appropriate rows
    X_train = X_train.loc[X_train['swing'] == 0]
    y_train = y_train.loc[X_train.index]

    # Drop filter feature
    X_train = X_train.drop(['swing'], axis = 1)
    X_test = X_test.drop(['swing'], axis = 1)

    # Train model on those rows
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X_train.shape[1],)),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(3, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    # Make predictions on entire test dataset
    preds = model.predict(X_test_scaled)

    strike_pred_all[test_idx] = preds[:, 0]
    ball_pred_all[test_idx] = preds[:, 1]
    hbp_pred_all[test_idx] = preds[:, 2]


df = df.with_columns(pl.Series("called_strike_pred_nn", strike_pred_all),
                               pl.Series("called_ball_pred_nn", ball_pred_all),
                               pl.Series("called_hbp_pred_nn", hbp_pred_all))

In [ ]:
df.write_csv('df_takes.csv')
files.download('df_takes.csv')

## Contact/Whiff Model

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_takes.csv')

In [ ]:
# xgboost sigmoid model
feats = ['swing'] + ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'contact'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

contact_pred_all = np.zeros_like(y, dtype=float)
whiff_pred_all = np.zeros_like(y, dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")
    # Split into train/test datasets
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    # Filter train dataset to only appropriate rows
    X_train = X_train.loc[X_train['swing'] == 1]
    y_train = y_train.loc[X_train.index]

    # Drop filter feature
    X_train = X_train.drop(['swing'], axis = 1)
    X_test = X_test.drop(['swing'], axis = 1)

    # Train model on those rows
    xgb = XGBClassifier()

    calibrated_xgb = CalibratedClassifierCV(xgb, method = 'sigmoid', cv = 3)
    calibrated_xgb.fit(X_train, y_train)

    # Make predictions on entire test dataset
    preds = calibrated_xgb.predict_proba(X_test)

    contact_pred_all[test_idx] = preds[:, 1]
    whiff_pred_all[test_idx] = preds[:, 0]


df = df.with_columns(pl.Series("contact_pred_xgb", contact_pred_all),
                               pl.Series("whiff_pred_xgb", whiff_pred_all))

In [ ]:
df.write_csv('df_contact.csv')
files.download('df_contact.csv')

## Batted Ball Model

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_contact.csv')

df = df.with_columns(
    pl.when(pl.col('bb_type') == 'foul')
    .then(0)
    .when(pl.col('bb_type') == 'ground_ball')
    .then(1)
    .when(pl.col('bb_type') == 'line_drive')
    .then(2)
    .when(pl.col('bb_type') == 'fly_ball')
    .then(3)
    .when(pl.col('bb_type') == 'popup')
    .then(4)
    .otherwise(5) # placeholder that will get dropped when model trains
    .alias('bb_target')
)

In [ ]:
# Neural net 3 layer model
feats = ['contact'] + ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'bb_target'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

foul_pred_all = np.zeros(y.shape[0], dtype=float)
gb_pred_all = np.zeros(y.shape[0], dtype=float)
ld_pred_all = np.zeros(y.shape[0], dtype=float)
fb_pred_all = np.zeros(y.shape[0], dtype=float)
po_pred_all = np.zeros(y.shape[0], dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")
    # Split into train/test datasets
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    # Filter train dataset to only appropriate rows
    X_train = X_train.loc[X_train['contact'] == 1]
    y_train = y_train.loc[X_train.index]

    # Drop filter feature
    X_train = X_train.drop(['contact'], axis = 1)
    X_test = X_test.drop(['contact'], axis = 1)

    # Train model on those rows
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X_train.shape[1],)),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    # Make predictions on entire test dataset
    preds = model.predict(X_test_scaled)

    foul_pred_all[test_idx] = preds[:, 0]
    gb_pred_all[test_idx] = preds[:, 1]
    ld_pred_all[test_idx] = preds[:, 2]
    fb_pred_all[test_idx] = preds[:, 3]
    po_pred_all[test_idx] = preds[:, 4]


df = df.with_columns(pl.Series("foul_pred_nn", foul_pred_all),
                               pl.Series("gb_pred_nn", gb_pred_all),
                               pl.Series("ld_pred_nn", ld_pred_all),
                               pl.Series("fb_pred_nn", fb_pred_all),
                               pl.Series("po_pred_nn", po_pred_all))

In [ ]:
df.write_csv('df_bb.csv')
files.download('df_bb.csv')

## Ground Ball Model

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_bb.csv')

df = df.with_columns(
    pl.when(pl.col('out') == 1)
    .then(0)
    .when(pl.col('single') == 1)
    .then(1)
    .when(pl.col('double') == 1)
    .then(2)
    .when(pl.col('triple') == 1)
    .then(3)
    .otherwise(4) # placeholder that gets dropped when filtered
    .alias('gb_target')
)

In [ ]:
# Neural net 4 layer model
feats = ['bb_type'] + ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'gb_target'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros(y.shape[0], dtype=float)
single_pred_all = np.zeros(y.shape[0], dtype=float)
double_pred_all = np.zeros(y.shape[0], dtype=float)
triple_pred_all = np.zeros(y.shape[0], dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    # Split into train/test datasets
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    # Filter train dataset to only appropriate rows
    X_train = X_train.loc[X_train['bb_type'] == 'ground_ball']
    y_train = y_train.loc[X_train.index]

    # Drop filter feature
    X_train = X_train.drop(['bb_type'], axis = 1)
    X_test = X_test.drop(['bb_type'], axis = 1)

    # Train model on filtered data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X_train.shape[1],)),
        Dense(128, activation = 'relu'),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(4, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    # Make predictions on entire test dataset
    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]


df = df.with_columns(pl.Series("gb_out_pred_nn", out_pred_all),
                               pl.Series("gb_single_pred_nn", single_pred_all),
                               pl.Series("gb_double_pred_nn", double_pred_all),
                               pl.Series("gb_triple_pred_nn", triple_pred_all))

In [ ]:
df.write_csv('df_gb.csv')
files.download('df_gb.csv')

## Line Drive Model

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_gb.csv')

df = df.with_columns(
    pl.when(pl.col('out') == 1)
    .then(0)
    .when(pl.col('single') == 1)
    .then(1)
    .when(pl.col('double') == 1)
    .then(2)
    .when(pl.col('triple') == 1)
    .then(3)
    .otherwise(4) # placeholder that gets dropped when filtered
    .alias('ld_target')
)

In [ ]:
# Neural net 2 layer model
feats = ['bb_type'] + ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'ld_target'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros(y.shape[0], dtype=float)
single_pred_all = np.zeros(y.shape[0], dtype=float)
double_pred_all = np.zeros(y.shape[0], dtype=float)
triple_pred_all = np.zeros(y.shape[0], dtype=float)
homerun_pred_all = np.zeros(y.shape[0], dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    # Split into train/test datasets
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    # Filter train dataset to only appropriate rows
    X_train = X_train.loc[X_train['bb_type'] == 'line_drive']
    y_train = y_train.loc[X_train.index]

    # Drop filter feature
    X_train = X_train.drop(['bb_type'], axis = 1)
    X_test = X_test.drop(['bb_type'], axis = 1)

    # Train model on filtered data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X_train.shape[1],)),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 1024, verbose = 1)

    # Make predictions on entire test dataset
    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]


df = df.with_columns(pl.Series("ld_out_pred_nn", out_pred_all),
                               pl.Series("ld_single_pred_nn", single_pred_all),
                               pl.Series("ld_double_pred_nn", double_pred_all),
                               pl.Series("ld_triple_pred_nn", triple_pred_all),
                           pl.Series("ld_homerun_pred_nn", homerun_pred_all))

In [ ]:
df = df.drop(['take_target', 'bb_target', 'gb_target', 'ld_target'])

In [ ]:
df.write_csv('df_ld.csv')
files.download('df_ld.csv')

## FB Model

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_ld.csv')

df = df.with_columns(
    pl.when(pl.col('out') == 1)
    .then(0)
    .when(pl.col('single') == 1)
    .then(1)
    .when(pl.col('double') == 1)
    .then(2)
    .when(pl.col('triple') == 1)
    .then(3)
    .otherwise(4) # placeholder that gets dropped when filtered
    .alias('fb_target')
)

In [ ]:
# Neural net 2 layer model
feats = ['bb_type'] + ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'fb_target'

df_pandas = df.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

out_pred_all = np.zeros(y.shape[0], dtype=float)
single_pred_all = np.zeros(y.shape[0], dtype=float)
double_pred_all = np.zeros(y.shape[0], dtype=float)
triple_pred_all = np.zeros(y.shape[0], dtype=float)
homerun_pred_all = np.zeros(y.shape[0], dtype=float)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    # Split into train/test datasets
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    # Filter train dataset to only appropriate rows
    X_train = X_train.loc[X_train['bb_type'] == 'fly_ball']
    y_train = y_train.loc[X_train.index]

    # Drop filter feature
    X_train = X_train.drop(['bb_type'], axis = 1)
    X_test = X_test.drop(['bb_type'], axis = 1)

    # Train model on filtered data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X_train.shape[1],)),
        Dense(32, activation = 'relu'),
        Dense(5, activation = 'softmax')
    ])

    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 1,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, callbacks = [early_stop], epochs = 15, batch_size = 64, verbose = 1)

    # Make predictions on entire dataset
    preds = model.predict(X_test_scaled)

    out_pred_all[test_idx] = preds[:, 0]
    single_pred_all[test_idx] = preds[:, 1]
    double_pred_all[test_idx] = preds[:, 2]
    triple_pred_all[test_idx] = preds[:, 3]
    homerun_pred_all[test_idx] = preds[:, 4]


df = df.with_columns(pl.Series("fb_out_pred_nn", out_pred_all),
                               pl.Series("fb_single_pred_nn", single_pred_all),
                               pl.Series("fb_double_pred_nn", double_pred_all),
                               pl.Series("fb_triple_pred_nn", triple_pred_all),
                           pl.Series("fb_homerun_pred_nn", homerun_pred_all))

## RV Preds

In [ ]:
# getting mean rv for each event
called_strike = df.filter(pl.col('called_strike') == 1)['delta_run_exp'].mean()
called_ball = df.filter(pl.col('called_ball') == 1)['delta_run_exp'].mean()
called_hbp = df.filter(pl.col('called_hbp') == 1)['delta_run_exp'].mean()

whiff = df.filter(pl.col('whiff') == 1)['delta_run_exp'].mean()

foul = df.filter(pl.col('foul') == 1)['delta_run_exp'].mean()
popup = df.filter(pl.col('bb_type') == 'popup')['delta_run_exp'].mean()

gb_out = df.filter((pl.col('bb_type') == 'ground_ball') & (pl.col('out') == 1))['delta_run_exp'].mean()
gb_single = df.filter((pl.col('bb_type') == 'ground_ball') & (pl.col('single') == 1))['delta_run_exp'].mean()
gb_double = df.filter((pl.col('bb_type') == 'ground_ball') & (pl.col('double') == 1))['delta_run_exp'].mean()
gb_triple = df.filter((pl.col('bb_type') == 'ground_ball') & (pl.col('triple') == 1))['delta_run_exp'].mean()

ld_out = df.filter((pl.col('bb_type') == 'line_drive') & (pl.col('out') == 1))['delta_run_exp'].mean()
ld_single = df.filter((pl.col('bb_type') == 'line_drive') & (pl.col('single') == 1))['delta_run_exp'].mean()
ld_double = df.filter((pl.col('bb_type') == 'line_drive') & (pl.col('double') == 1))['delta_run_exp'].mean()
ld_triple = df.filter((pl.col('bb_type') == 'line_drive') & (pl.col('triple') == 1))['delta_run_exp'].mean()
ld_homerun = df.filter((pl.col('bb_type') == 'line_drive') & (pl.col('home_run') == 1))['delta_run_exp'].mean()

fb_out = df.filter((pl.col('bb_type') == 'fly_ball') & (pl.col('out') == 1))['delta_run_exp'].mean()
fb_single = df.filter((pl.col('bb_type') == 'fly_ball') & (pl.col('single') == 1))['delta_run_exp'].mean()
fb_double = df.filter((pl.col('bb_type') == 'fly_ball') & (pl.col('double') == 1))['delta_run_exp'].mean()
fb_homerun = df.filter((pl.col('bb_type') == 'fly_ball') & (pl.col('home_run') == 1))['delta_run_exp'].mean()

In [ ]:
# generating rv preds
df = df.with_columns(
    xRV_model1 = pl.col('take_pred_xgb')*pl.col('called_strike_pred_nn')*called_strike +
    pl.col('take_pred_xgb')*pl.col('called_ball_pred_nn')*called_ball +
    pl.col('take_pred_xgb')*pl.col('called_hbp_pred_nn')*called_hbp +

    pl.col('swing_pred_xgb')*pl.col('whiff_pred_xgb')*whiff +

    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('foul_pred_nn')*foul +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('po_pred_nn')*popup +

    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('gb_pred_nn')*pl.col('gb_out_pred_nn')*gb_out +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('gb_pred_nn')*pl.col('gb_single_pred_nn')*gb_single +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('gb_pred_nn')*pl.col('gb_double_pred_nn')*gb_double +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('gb_pred_nn')*pl.col('gb_triple_pred_nn')*gb_triple +

    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('ld_pred_nn')*pl.col('ld_out_pred_nn')*ld_out +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('ld_pred_nn')*pl.col('ld_single_pred_nn')*ld_single +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('ld_pred_nn')*pl.col('ld_double_pred_nn')*ld_double +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('ld_pred_nn')*pl.col('ld_triple_pred_nn')*ld_triple +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('ld_pred_nn')*pl.col('ld_homerun_pred_nn')*ld_homerun +

    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('fb_pred_nn')*pl.col('fb_out_pred_nn')*fb_out +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('fb_pred_nn')*pl.col('fb_single_pred_nn')*fb_single +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('fb_pred_nn')*pl.col('fb_double_pred_nn')*fb_double +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('fb_pred_nn')*pl.col('fb_triple_pred_nn')*fb_triple +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('fb_pred_nn')*pl.col('fb_homerun_pred_nn')*fb_homerun
)

In [ ]:
df.write_csv('df_model1.csv')
files.download('df_model1.csv')

In [ ]:
corr_valid(metric = 'xRV_model1', model = 'Ensemble Model 1')

In [ ]:
sns.histplot(df.select(pl.col('xRV_model1')))
plt.xlim([-0.05, 0.05])
plt.show()

In [ ]:
# best by year
df.group_by(pl.col('game_year'), pl.col('player_name')).agg(pl.col('xRV_model1').mean()).sort('xRV_model1', descending = False).head(10)

In [ ]:
# best by yearly pitch
 (df.with_columns([
        pl.len().over(["player_name", "pitch_type"]).alias("pitch_count")
    ])
    .filter(pl.col("pitch_count") >= 100)
    .group_by(pl.col('game_year'), pl.col('player_name'), pl.col('pitch_type'))
    .agg(pl.col('xRV_model1').mean())
    .sort('xRV_model1', descending = False).head(10)
)

# Bayes Launch Angle

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_model1.csv')

In [ ]:
# drop unused data
drop = ['popup', 'fb_pred_nn', 'fb_single_pred_nn', 'stand', 'api_break_x_arm', 'gb_triple_pred_nn', 'called_ball', 'sz_top', 'gb_double_pred_nn',
    'home_run', 'fb_double_pred_nn', 'delta_home_win_exp', 'ld_triple_pred_nn', 'fly_ball', 'events', 'balls', 'gb_out_pred_nn', 'ld_pred_nn', 'adj_vx0',
    'ld_homerun_pred_nn', 'out', 'batter', 'whiff', 'api_break_z_with_gravity', 'p_throws', 'ld_double_pred_nn', 'vz0', 'called_hbp', 'swing', 'pitcher',
    'primary_fb_speed', 'fb_out_pred_nn', 'game_date', 'estimated_woba_using_speedangle', 'ground_ball', 'line_drive', 'triple', 'ld_out_pred_nn', 'bb_type',
    'foul_pred_nn', 'primary_fb_horz', 'po_pred_nn', 'vy0', 'plate_z', 'plate_x', 'primary_fb_vert',
    'double', 'gb_pred_nn', 'strikes', 'woba_value', 'gb_single_pred_nn', 'fb_homerun_pred_nn', 'single', 'sz_bot',
    'ld_single_pred_nn', 'fb_triple_pred_nn', 'called_strike', 'fb_target', 'vx0']

# filter to 2023-2024
df_bayes = df.drop(drop).filter(pl.col('game_year').is_in([2023, 2024]))
# filter to pitches thrown >750 times in a season by a pitcher
df_qual = df_bayes.group_by(['game_year', 'player_name', 'pitch_type']).len().filter(pl.col('len') > 750).drop('len')
df_bayes = df_bayes.join(df_qual, on = ['game_year', 'player_name', 'pitch_type'], how = 'inner')

In [ ]:
df_bayes.shape

In [ ]:
df_bayes.write_csv('df_bayes.csv')
files.download('df_bayes.csv')

In [ ]:
df_bayes = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_bayes.csv')

In [ ]:
# filter to relevant rows
df_bip = df_bayes.filter(pl.col('description') == 'hit_into_play')
df_bip = df_bip.with_row_index()

In [ ]:
# sample dfs for faster plotting
df_bip_r = df_bip.filter(pl.col('righty') == 1).sample(10000)
df_bip_l = df_bip.filter(pl.col('righty') == 0).sample(10000)

feats = ['release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed',
         'release_spin_rate', 'spin_axis', 'pfx_x', 'ax', 'pfx_z', 'az']

# correlation plots for potential feats
for feat in feats:
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,4))

    sns.regplot(x = df_bip_r[feat], y = df_bip_r['launch_angle'], ax = ax1)
    corr_r = round(df_bip_r.select(pl.corr(feat, 'launch_angle', method = 'pearson')).item(), 4)
    ax1.set_title(f'Righties {feat} vs launch angle \nCorr: {corr_r}')
    ax1.set_ylim([-110, 110])

    sns.regplot(x = df_bip_l[feat], y = df_bip_l['launch_angle'], ax = ax2)
    corr_l = round(df_bip_l.select(pl.corr(feat, 'launch_angle', method = 'pearson')).item(), 4)
    ax2.set_title(f'Lefties {feat} vs launch angle \nCorr: {corr_l}')
    ax2.set_ylim([-110, 110])

    plt.show()

In [ ]:
sns.histplot(df_bip['launch_angle'], bins = 30)

## Just Vert

In [ ]:
# Variables
obs_id = np.arange(len(df_bip))
X_data = df_bip.select('pfx_z').to_numpy().flatten()
launch_angle = df_bip.select('launch_angle').to_numpy().flatten()

coords = {'obs_id': obs_id}
coords['metric'] = 'launch_angle'

with pm.Model(coords = coords) as model_la:

    # Priors
    alpha = pm.Normal("alpha", mu=10, sigma=10)
    beta = pm.Normal("beta", mu=2.5, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=15)

    # Expected value
    mu = alpha + beta * X_data

    # Likelihood
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=launch_angle, dims="obs_id")

In [ ]:
pm.model_to_graphviz(model_la)

In [ ]:
with model_la:
    # Prior predictive check
    model_la_trace = pm.sample_prior_predictive(samples = 100)

In [ ]:
sns.histplot(model_la_trace.prior['beta'].squeeze(), bins = 20)
plt.xlabel('Launch Angle Beta')
plt.title('Launch Angle Beta Prior Distribution')
plt.show()

In [ ]:
az.plot_ppc(model_la_trace, group='prior')

In [ ]:
sns.histplot(model_la_trace.prior_predictive['y_obs'].squeeze().values.flatten(), bins = 135)
plt.xlabel('Launch Angle')
plt.title('Launch Angle Prior Predictive Distribution')
plt.xlim([-100, 100])
plt.show()

In [ ]:
# inference
with model_la:
    trace = pm.sample(draws=1000, tune=1000, cores=4, chains=4, random_seed=42)

In [ ]:
az.summary(trace).sort_values('r_hat', ascending=False).head()

In [ ]:
model_la_trace.extend(trace)

In [ ]:
sns.histplot(model_la_trace.posterior['beta'].squeeze().values.flatten(), bins = 15)
plt.xlabel('Launch Angle Beta')
plt.title('Launch Angle Beta Posterior Distribution')
plt.show()

In [ ]:
az.plot_trace(model_la_trace, var_names = 'beta')

In [ ]:
# posterior preds
with model_la:
    posterior_predictive = pm.sample_posterior_predictive(model_la_trace, extend_inferencedata=True)

In [ ]:
with open('/content/drive/MyDrive/Datasets/stuff/model_la_trace.pkl', 'wb') as file:
    pickle.dump(model_la_trace, file)

In [ ]:
with open('/content/drive/MyDrive/Datasets/stuff/model_la_trace.pkl', 'rb') as f:
    model_la_trace = pickle.load(f)

In [ ]:
az.plot_ppc(posterior_predictive, group='posterior')

In [ ]:
with model_la:
    pm.compute_log_likelihood(model_la_trace)

az.loo(model_la_trace, model_la)

## Multiple Variables

In [ ]:
# Variables
obs_id = np.arange(len(df_bip))
df_bip = df_bip.with_columns(
    pfx_z_norm = (pl.col('pfx_z') - pl.col('pfx_z').mean())/pl.col('pfx_z').std(),
    adj_pfx_x_norm = (pl.col('adj_pfx_x') - pl.col('adj_pfx_x').mean())/pl.col('adj_pfx_x').std(),
    release_spin_rate_norm = (pl.col('release_spin_rate') - pl.col('release_spin_rate').mean())/pl.col('release_spin_rate').std()
)
pfx_z_norm = df_bip.select('pfx_z_norm').to_numpy().flatten()
adj_pfx_x_norm = df_bip.select('adj_pfx_x_norm').to_numpy().flatten()
release_spin_rate_norm = df_bip.select('release_spin_rate_norm').to_numpy().flatten()
launch_angle = df_bip.select('launch_angle').to_numpy().flatten()

coords = {'obs_id': obs_id}

with pm.Model(coords = coords) as model_la2:

    # Priors
    alpha = pm.Normal("alpha", mu=5, sigma=5)
    beta_pfx_z_norm = pm.Normal("beta_pfx_z_norm", mu=0, sigma=10)
    beta_adj_pfx_x_norm = pm.Normal("beta_adj_pfx_x_norm", mu=5, sigma=10)
    beta_release_spin_rate_norm = pm.Normal("beta_release_spin_rate_norm", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=3)

    # Expected value
    mu = alpha + beta_pfx_z_norm*pfx_z_norm + beta_adj_pfx_x_norm*adj_pfx_x_norm + beta_release_spin_rate_norm*release_spin_rate_norm
    # Likelihood
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=launch_angle, dims="obs_id")

In [ ]:
pm.model_to_graphviz(model_la2)

In [ ]:
with model_la2:
    # Prior predictive check
    model_la2_trace = pm.sample_prior_predictive(samples = 100)

In [ ]:
sns.histplot(model_la2_trace.prior_predictive['y_obs'].squeeze().values.flatten())
plt.xlabel('Launch Angle')
plt.title('Launch Angle Prior Predictive Distribution')
plt.xlim([-100, 100])
plt.show()

In [ ]:
# inference
with model_la2:
    trace = pm.sample(draws=1000, tune=1000, cores=4, chains=4, random_seed=42)

In [ ]:
az.summary(trace).sort_values('r_hat', ascending=False).head()

In [ ]:
model_la2_trace.extend(trace)

In [ ]:
az.plot_trace(model_la2_trace, var_names = ['beta_adj_pfx_x_norm', 'beta_pfx_z_norm', 'beta_release_spin_rate_norm'], figsize=(12, 10))

In [ ]:
# posterior predictive
with model_la2:
    posterior_predictive = pm.sample_posterior_predictive(model_la2_trace, extend_inferencedata=True)

In [ ]:
az.plot_ppc(posterior_predictive, group='posterior')

In [ ]:
with model_la2:
    pm.compute_log_likelihood(model_la2_trace)

az.loo(model_la2_trace, model_la2)

## Preds

In [ ]:
X_cols = ["pfx_z", "adj_pfx_x", "release_spin_rate", 'description']
y_col = "launch_angle"

X = df_bayes.select(X_cols)
y = df_bayes.select(y_col)

# To pandas
X = X.to_pandas()
y = y.to_pandas()

kf = KFold(n_splits=5, shuffle=True, random_state=42)
xLA_25_all = np.zeros(y.shape[0])
xLA_50_all = np.zeros(y.shape[0])
xLA_75_all = np.zeros(y.shape[0])

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    # Create train/test splits
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Filter train data to BIP
    X_train = X_train.loc[X_train['description'] == 'hit_into_play']
    y_train = y_train.loc[y_train.index.isin(X_train.index)]

    # Drop filter feature
    X_train = X_train.drop(['description'], axis = 1)
    X_test = X_test.drop(['description'], axis = 1)

    # To polars
    X_train = pl.DataFrame(X_train)
    X_test = pl.DataFrame(X_test)
    y_train = pl.DataFrame(y_train)
    y_test = pl.DataFrame(y_test)

    # Train Variables
    pfx_z_mean_train = X_train.select('pfx_z').mean().item()
    pfx_z_std_train = X_train.select('pfx_z').std().item()
    adj_pfx_x_mean_train = X_train.select('adj_pfx_x').mean().item()
    adj_pfx_x_std_train = X_train.select('adj_pfx_x').std().item()
    release_spin_rate_mean_train = X_train.select('release_spin_rate').mean().item()
    release_spin_rate_std_train = X_train.select('release_spin_rate').std().item()

    X_train = X_train.with_columns(
        pfx_z_norm = (pl.col('pfx_z') - pl.col('pfx_z').mean())/pl.col('pfx_z').std(),
        adj_pfx_x_norm = (pl.col('adj_pfx_x') - pl.col('adj_pfx_x').mean())/pl.col('adj_pfx_x').std(),
        release_spin_rate_norm = (pl.col('release_spin_rate') - pl.col('release_spin_rate').mean())/pl.col('release_spin_rate').std()
    )
    pfx_z_norm_train = X_train.select('pfx_z_norm').to_numpy().flatten()
    adj_pfx_x_norm_train = X_train.select('adj_pfx_x_norm').to_numpy().flatten()
    release_spin_rate_norm_train = X_train.select('release_spin_rate_norm').to_numpy().flatten()
    launch_angle_train = y_train.to_numpy().flatten()

    # Test Variables
    X_test = X_test.with_columns(
        pfx_z_norm = (pl.col('pfx_z') - pfx_z_mean_train)/pfx_z_std_train,
        adj_pfx_x_norm = (pl.col('adj_pfx_x') - adj_pfx_x_mean_train)/adj_pfx_x_std_train,
        release_spin_rate_norm = (pl.col('release_spin_rate') - release_spin_rate_mean_train)/release_spin_rate_std_train
    )
    pfx_z_norm_test = X_test.select('pfx_z_norm').to_numpy().flatten()
    adj_pfx_x_norm_test = X_test.select('adj_pfx_x_norm').to_numpy().flatten()
    release_spin_rate_norm_test = X_test.select('release_spin_rate_norm').to_numpy().flatten()
    launch_angle_test = y_test.to_numpy().flatten()


    with pm.Model() as model_la_pred:
        # Declare data
        pfx_z_norm = pm.Data('pfx_z_norm', pfx_z_norm_train)
        adj_pfx_x_norm = pm.Data('adj_pfx_x_norm', adj_pfx_x_norm_train)
        release_spin_rate_norm = pm.Data('release_spin_rate_norm', release_spin_rate_norm_train)
        launch_angle = pm.Data('launch_angle', launch_angle_train)

        # Priors
        alpha = pm.Normal("alpha", mu=5, sigma=5)
        beta_pfx_z_norm = pm.Normal("beta_pfx_z_norm", mu=0, sigma=10)
        beta_adj_pfx_x_norm = pm.Normal("beta_adj_pfx_x_norm", mu=5, sigma=10)
        beta_release_spin_rate_norm = pm.Normal("beta_release_spin_rate_norm", mu=0, sigma=10)
        sigma = pm.HalfNormal("sigma", sigma=3)

        # Expected value
        mu = alpha + beta_pfx_z_norm*pfx_z_norm + beta_adj_pfx_x_norm*adj_pfx_x_norm + beta_release_spin_rate_norm*release_spin_rate_norm
        # Likelihood
        y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=launch_angle)

        # Sample posterior
        trace = pm.sample(1000, tune=1000, cores=4, chains=4, random_seed=42)

        # Predict on test set
        pm.set_data({
        "pfx_z_norm": pfx_z_norm_test,
        "adj_pfx_x_norm": adj_pfx_x_norm_test,
        "release_spin_rate_norm": release_spin_rate_norm_test,
        'launch_angle': launch_angle_test
        })

        posterior_predictive = pm.sample_posterior_predictive(trace, model = model_la_pred, var_names = ['y_obs'])

        # Get percentiles
        y_pred_samples = posterior_predictive.posterior_predictive["y_obs"]
        n_y_obs = y_pred_samples.shape[-1]
        flat_samples = y_pred_samples.values.reshape(-1, n_y_obs)

        percentiles = np.percentile(flat_samples, [25, 50, 75], axis=0)
        xLA_25_all[test_idx] = percentiles[0]
        xLA_50_all[test_idx] = percentiles[1]
        xLA_75_all[test_idx] = percentiles[2]

In [ ]:
df_bayes = df_bayes.with_columns(pl.Series("xLA_25", xLA_25_all),
                               pl.Series("xLA_50", xLA_50_all),
                               pl.Series("xLA_75", xLA_75_all))

In [ ]:
df_bayes.head()

In [ ]:
df_bayes.write_csv('df_bayes_la.csv')
files.download('df_bayes_la.csv')

# Bayes Exit Velo

In [ ]:
df_bayes_la = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_bayes_la.csv')

In [ ]:
# filter to relevant rows
df_bip = df_bayes_la.filter(pl.col('description') == 'hit_into_play')
df_bip = df_bip.with_row_index()

In [ ]:
# sample dfs for faster plotting
df_bip_r = df_bip.filter(pl.col('p_throws') == 'R').sample(10000)
df_bip_l = df_bip.filter(pl.col('p_throws') == 'L').sample(10000)

feats = ['release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed',
         'release_spin_rate', 'spin_axis', 'pfx_x', 'ax', 'pfx_z', 'az']

# corr plots for potential feats
for feat in feats:
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,4))

    sns.regplot(x = df_bip_r[feat], y = df_bip_r['launch_speed'], ax = ax1)
    corr_r = round(df_bip_r.select(pl.corr(feat, 'launch_speed', method = 'pearson')).item(), 4)
    ax1.set_title(f'Righties {feat} vs exit velocity \nCorr: {corr_r}')
    ax1.set_ylim([5, 125])

    sns.regplot(x = df_bip_l[feat], y = df_bip_l['launch_speed'], ax = ax2)
    corr_l = round(df_bip_l.select(pl.corr(feat, 'launch_speed', method = 'pearson')).item(), 4)
    ax2.set_title(f'Lefties {feat} vs exit velocity \nCorr: {corr_l}')
    ax2.set_ylim([5, 125])

    plt.show()

## Just Velo

In [ ]:
sns.histplot(df_bip['launch_speed'])
plt.title('Exit Velo Distribution')
plt.show()

In [ ]:
# Variables
df_bip = df_bip.with_columns(
    release_speed_norm = (pl.col('release_speed') - pl.col('release_speed').mean())/pl.col('release_speed').std()
)
release_speed_norm = df_bip.select('release_speed_norm').to_numpy().flatten()
launch_speed = df_bip.select('launch_speed').to_numpy().flatten()

with pm.Model() as model_ev:

    # Priors
    theta = pm.Normal('theta', mu = 80, sigma = 10)
    beta = pm.Normal("beta", mu = 5, sigma = 10)
    alpha = pm.Normal("alpha", mu = -6, sigma = 3)
    sigma = pm.HalfNormal("sigma", sigma=0.75)

    # Expected value
    mu = theta + beta * release_speed_norm

    # Likelihood
    y_obs = pm.SkewNormal("y_obs", mu=mu, sigma=sigma, alpha = alpha, observed=launch_speed)

In [ ]:
pm.model_to_graphviz(model_ev)

In [ ]:
with model_ev:
    # Prior predictive check
    model_ev_trace = pm.sample_prior_predictive(samples = 100)

In [ ]:
az.plot_ppc(model_ev_trace, group='prior')

In [ ]:
sns.histplot(model_ev_trace.prior_predictive['y_obs'].squeeze().values.flatten(), bins = 135)
plt.xlabel('Exit Velo')
plt.title('Exit Velo Prior Predictive Distribution')
plt.show()

In [ ]:
# inference
with model_ev:
    trace = pm.sample(draws=1000, tune=1000, cores=4, chains=4, random_seed=42)

In [ ]:
az.summary(trace).sort_values('r_hat', ascending=False).head()

In [ ]:
model_ev_trace.extend(trace)

In [ ]:
az.plot_trace(model_ev_trace, var_names = ['alpha', 'beta', 'sigma', 'theta'], figsize=(12, 10))

In [ ]:
# posterior predictive
with model_ev:
    posterior_predictive_ev = pm.sample_posterior_predictive(model_ev_trace, extend_inferencedata=True)

In [ ]:
with open('/content/drive/MyDrive/Datasets/stuff/model_ev_trace.pkl', 'wb') as file:
    pickle.dump(posterior_predictive_ev, file)

In [ ]:
with open('/content/drive/MyDrive/Datasets/stuff/model_ev_trace.pkl', 'rb') as f:
    posterior_predictive_ev = pickle.load(f)

In [ ]:
az.plot_ppc(posterior_predictive_ev, group='posterior')

In [ ]:
with model_ev:
    pm.compute_log_likelihood(model_ev_trace)

az.loo(model_ev_trace, model_ev)

## More Variables

In [ ]:
# Variables
df_bip = df_bip.with_columns(
    release_speed_norm = (pl.col('release_speed') - pl.col('release_speed').mean())/pl.col('release_speed').std(),
    adj_ax_norm = (pl.col('adj_ax') - pl.col('adj_ax').mean())/pl.col('adj_ax').std(),
    az_norm = (pl.col('az') - pl.col('az').mean())/pl.col('az').std()
)
release_speed_norm = df_bip.select('release_speed_norm').to_numpy().flatten()
adj_ax_norm = df_bip.select('adj_ax_norm').to_numpy().flatten()
az_norm = df_bip.select('az_norm').to_numpy().flatten()
launch_speed = df_bip.select('launch_speed').to_numpy().flatten()

with pm.Model() as model_ev2:

    # Priors
    theta = pm.Normal('theta', mu = 80, sigma = 10)
    beta_release_speed_norm = pm.Normal("beta_release_speed_norm", mu = 5, sigma = 10)
    beta_adj_ax_norm = pm.Normal("beta_adj_ax_norm", mu = -5, sigma = 10)
    beta_az_norm = pm.Normal("beta_az_norm", mu = 5, sigma = 10)
    alpha = pm.Normal("alpha", mu = -6, sigma = 3)
    sigma = pm.HalfNormal("sigma", sigma=0.75)

    # Expected value
    mu = theta + beta_release_speed_norm*release_speed_norm + beta_adj_ax_norm*adj_ax_norm + beta_az_norm*az_norm

    # Likelihood
    y_obs = pm.SkewNormal("y_obs", mu=mu, sigma=sigma, alpha = alpha, observed=launch_speed)

In [ ]:
pm.model_to_graphviz(model_ev2)

In [ ]:
with model_ev2:
    # Prior predictive check
    model_ev2_trace = pm.sample_prior_predictive(samples = 100)

In [ ]:
sns.histplot(model_ev2_trace.prior_predictive['y_obs'].squeeze().values.flatten())
plt.xlabel('Exit Velo')
plt.title('Exit Velo Prior Predictive Distribution')
plt.show()

In [ ]:
# inference
with model_ev2:
    trace = pm.sample(draws=1000, tune=1000, cores=4, chains=4, random_seed=42)

In [ ]:
az.summary(trace).sort_values('r_hat', ascending=False).head()

In [ ]:
model_ev2_trace.extend(trace)

In [ ]:
az.plot_trace(model_ev2_trace, var_names = ['beta_release_speed_norm', 'beta_release_speed_norm', 'beta_az_norm'], figsize=(12, 10))

In [ ]:
with open('/content/drive/MyDrive/Datasets/stuff/model_ev2_trace.pkl', 'wb') as file:
    pickle.dump(model_ev2_trace, file)

In [ ]:
with open('/content/drive/MyDrive/Datasets/stuff/model_ev2_trace.pkl', 'rb') as f:
    model_ev2_trace = pickle.load(f)

In [ ]:
# posterior predictive
with model_ev2:
    posterior_predictive_ev = pm.sample_posterior_predictive(model_ev2_trace, extend_inferencedata=True)

In [ ]:
az.plot_ppc(posterior_predictive_ev, group='posterior')

In [ ]:
with model_ev2:
    pm.compute_log_likelihood(model_ev2_trace)

az.loo(model_ev2_trace, model_ev2)

## Preds

In [ ]:
X_cols = ["release_speed", "adj_ax", "az", 'description']
y_col = "launch_speed"

X = df_bayes_la.select(X_cols)
y = df_bayes_la.select(y_col)

X = X.to_pandas()
y = y.to_pandas()

kf = KFold(n_splits=5, shuffle=True, random_state=42)
xEV_25_all = np.zeros(y.shape[0])
xEV_50_all = np.zeros(y.shape[0])
xEV_75_all = np.zeros(y.shape[0])

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold + 1}...")

    # Create train/test splits
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Filter train data to BIP
    X_train = X_train.loc[X_train['description'] == 'hit_into_play']
    y_train = y_train.loc[y_train.index.isin(X_train.index)]

    # Drop filter feature
    X_train = X_train.drop(['description'], axis = 1)
    X_test = X_test.drop(['description'], axis = 1)

    # To polars
    X_train = pl.DataFrame(X_train)
    X_test = pl.DataFrame(X_test)
    y_train = pl.DataFrame(y_train)
    y_test = pl.DataFrame(y_test)

    # Train Variables
    release_speed_mean_train = X_train.select('release_speed').mean().item()
    release_speed_std_train = X_train.select('release_speed').std().item()
    adj_ax_mean_train = X_train.select('adj_ax').mean().item()
    adj_ax_std_train = X_train.select('adj_ax').std().item()
    az_mean_train = X_train.select('az').mean().item()
    az_std_train = X_train.select('az').std().item()

    X_train = X_train.with_columns(
        release_speed_norm = (pl.col('release_speed') - pl.col('release_speed').mean())/pl.col('release_speed').std(),
        adj_ax_norm = (pl.col('adj_ax') - pl.col('adj_ax').mean())/pl.col('adj_ax').std(),
        az_norm = (pl.col('az') - pl.col('az').mean())/pl.col('az').std()
    )
    release_speed_norm_train = X_train.select('release_speed_norm').to_numpy().flatten()
    adj_ax_norm_train = X_train.select('adj_ax_norm').to_numpy().flatten()
    az_norm_train = X_train.select('az_norm').to_numpy().flatten()
    launch_speed_train = y_train.to_numpy().flatten()

    # Test Variables
    X_test = X_test.with_columns(
        release_speed_norm = (pl.col('release_speed') - release_speed_mean_train)/release_speed_std_train,
        adj_ax_norm = (pl.col('adj_ax') - adj_ax_mean_train)/adj_ax_std_train,
        az_norm = (pl.col('az') - az_mean_train)/az_std_train
    )
    release_speed_norm_test = X_test.select('release_speed_norm').to_numpy().flatten()
    adj_ax_norm_test = X_test.select('adj_ax_norm').to_numpy().flatten()
    az_norm_test = X_test.select('az_norm').to_numpy().flatten()
    launch_speed_test = y_test.to_numpy().flatten()


    with pm.Model() as model_ev_pred:
        # Declare data
        release_speed_norm = pm.Data('release_speed_norm', release_speed_norm_train)
        adj_ax_norm = pm.Data('adj_ax_norm', adj_ax_norm_train)
        az_norm = pm.Data('az_norm', az_norm_train)
        launch_speed = pm.Data('launch_speed', launch_speed_train)

        # Priors
        theta = pm.Normal('theta', mu = 80, sigma = 10)
        beta_release_speed_norm = pm.Normal("beta_release_speed_norm", mu = 5, sigma = 10)
        beta_adj_ax_norm = pm.Normal("beta_adj_ax_norm", mu = -5, sigma = 10)
        beta_az_norm = pm.Normal("beta_az_norm", mu = 5, sigma = 10)
        alpha = pm.Normal("alpha", mu = -6, sigma = 3)
        sigma = pm.HalfNormal("sigma", sigma=0.75)

        # Expected value
        mu = theta + beta_release_speed_norm*release_speed_norm + beta_adj_ax_norm*adj_ax_norm + beta_az_norm*az_norm

        # Likelihood
        y_obs = pm.SkewNormal("y_obs", mu=mu, sigma=sigma, alpha = alpha, observed=launch_speed)

        # Sample posterior
        trace = pm.sample(1000, tune=1000, cores=4, chains=4, random_seed=42)

        # Predict on test set
        pm.set_data({
        "release_speed_norm": release_speed_norm_test,
        "adj_ax_norm": adj_ax_norm_test,
        "az_norm": az_norm_test,
        'launch_speed': launch_speed_test
        })

        posterior_predictive = pm.sample_posterior_predictive(trace, model = model_ev_pred, var_names = ['y_obs'])

        # Get percentiles
        y_pred_samples = posterior_predictive.posterior_predictive["y_obs"]
        n_y_obs = y_pred_samples.shape[-1]
        flat_samples = y_pred_samples.values.reshape(-1, n_y_obs)

        percentiles = np.percentile(flat_samples, [25, 50, 75], axis=0)
        xEV_25_all[test_idx] = percentiles[0]
        xEV_50_all[test_idx] = percentiles[1]
        xEV_75_all[test_idx] = percentiles[2]

In [ ]:
df_bayes_la = df_bayes_la.with_columns(pl.Series("xEV_25", xEV_25_all),
                               pl.Series("xEV_50", xEV_50_all),
                               pl.Series("xEV_75", xEV_75_all))

# Foul/No Foul Model

In [ ]:
# xgboost model
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'foul'

df_pandas = df_bayes_la.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBClassifier()
preds = cross_val_predict(model, X, y , cv=5, method='predict_proba')

df_bayes_la = df_bayes_la.with_columns(pl.Series("foul_pred_xgb", preds[:, 1]),
                     pl.Series("no_foul_pred_xgb", preds[:, 0]))

In [ ]:
calibration_plot(n_bins = 10, data = df_bayes_la, preds = 'foul_pred_xgb', actuals = 'foul', model = 'XGBoost Foul')

In [ ]:
# Neural net 2 layers
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'foul'

df_pandas = df_bayes_la.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

foul_pred_all = np.zeros(y.shape[0], dtype=float)
no_foul_pred_all = np.zeros(y.shape[0], dtype=float)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Input(shape = (X.shape[1], )),
        Dense(32, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ])
    model.compile(optimizer = Adam(), loss = 'binary_crossentropy')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 0,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, batch_size = 1024, epochs = 15, callbacks = [early_stop], verbose = 1)

    y_pred = model.predict(X_test_scaled).flatten()

    foul_pred_all[test_idx] = y_pred

df_bayes_la = df_bayes_la.with_columns(pl.Series("foul_pred_nn", foul_pred_all))

In [ ]:
calibration_plot(n_bins = 10, data = df_bayes_la, preds = 'foul_pred_nn', actuals = 'foul', model = 'Neural Net 2 Layers Foul')

In [ ]:
df_bayes_la = df_bayes_la.drop(['foul_pred_xgb', 'no_foul_pred_xgb'])

In [ ]:
df_bayes_la.write_csv('df_bayes_la.csv')
files.download('df_bayes_la.csv')

# xRV BIP Model

In [ ]:
df_bayes_ev = pl.read_csv('df_bayes_la.csv')

In [ ]:
# xgboost model
feats = ['launch_speed', 'launch_angle']
target = 'delta_run_exp'

df_pandas = df_bip.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBRegressor()
preds = cross_val_predict(model, X, y , cv=5)

df_bip = df_bip.with_columns(pl.Series("xRV_xgb", preds))

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(df_bip.select('delta_run_exp'), df_bip.select('xRV_xgb')))

sns.regplot(data=df_bip, x = 'xRV_xgb', y = 'delta_run_exp')
plt.xlabel('Preds')
plt.ylabel('Actual')
plt.title(f'XGBoost xRV Validation \nRMSE: {round(rmse, 4)}')
plt.show()

In [ ]:
# Neural net 4 layers
feats = ['launch_speed', 'launch_angle']
target = 'delta_run_exp'

df_pandas = df_bip.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]

y_pred_all = np.zeros(y.shape[0], dtype=float)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"Training fold {fold + 1}...")

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test =  y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = Sequential([
        Dense(128, activation = 'relu', input_shape = (X.shape[1], )),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(1)
    ])
    model.compile(optimizer = Adam(learning_rate = 0.0005), loss = 'mse')

    early_stop = EarlyStopping(
        monitor = 'val_loss',
        patience = 2,
        restore_best_weights = True,
        verbose = 1
    )

    model.fit(X_train_scaled, y_train, validation_split = 0.2, batch_size = 1024, epochs = 15, callbacks = [early_stop], verbose = 1)

    y_pred = model.predict(X_test_scaled).flatten()

    y_pred_all[test_idx] = y_pred

df_bip = df_bip.with_columns(pl.Series("xRV_pred_nn_4", y_pred_all))

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(df_bip.select('delta_run_exp'), df_bip.select('xRV_pred_nn_4')))

sns.regplot(data=df_bip, x = 'xRV_pred_nn_4', y = 'delta_run_exp')
plt.xlabel('Preds')
plt.ylabel('Actual')
plt.title(f'Neural Net 4 Layers xRV Validation \nRMSE: {round(rmse, 4)}')
plt.show()

In [ ]:
# fit xRV model
feats = ['launch_speed', 'launch_angle']
target = 'delta_run_exp'

df_bip = df_bayes_ev.filter(pl.col('description') == 'hit_into_play')
df_pandas_fit = df_bip.to_pandas()
df_pandas_predict = df_bayes_ev.to_pandas()

model = XGBRegressor()
model.fit(df_pandas_fit[feats], df_pandas_fit[target])

In [ ]:
df_pandas_predict = df_bayes_ev.to_pandas()

# 25th percentile launch angles
df_pandas_predict_la25_ev25 = df_pandas_predict[['xLA_25', 'xEV_25']]
df_pandas_predict_la25_ev25 = df_pandas_predict_la25_ev25.rename(columns = {'xLA_25': 'launch_angle', 'xEV_25': 'launch_speed'})
preds_la25_ev25 = model.predict(df_pandas_predict_la25_ev25[feats])

df_pandas_predict_la25_ev50 = df_pandas_predict[['xLA_25', 'xEV_50']]
df_pandas_predict_la25_ev50 = df_pandas_predict_la25_ev50.rename(columns = {'xLA_25': 'launch_angle', 'xEV_50': 'launch_speed'})
preds_la25_ev50 = model.predict(df_pandas_predict_la25_ev50[feats])

df_pandas_predict_la25_ev75 = df_pandas_predict[['xLA_25', 'xEV_75']]
df_pandas_predict_la25_ev75 = df_pandas_predict_la25_ev75.rename(columns = {'xLA_25': 'launch_angle', 'xEV_75': 'launch_speed'})
preds_la25_ev75 = model.predict(df_pandas_predict_la25_ev75[feats])

# 50th percentile launch angles
df_pandas_predict_la50_ev25 = df_pandas_predict[['xLA_50', 'xEV_25']]
df_pandas_predict_la50_ev25 = df_pandas_predict_la50_ev25.rename(columns = {'xLA_50': 'launch_angle', 'xEV_25': 'launch_speed'})
preds_la50_ev25 = model.predict(df_pandas_predict_la50_ev25[feats])

df_pandas_predict_la50_ev50 = df_pandas_predict[['xLA_50', 'xEV_50']]
df_pandas_predict_la50_ev50 = df_pandas_predict_la50_ev50.rename(columns = {'xLA_50': 'launch_angle', 'xEV_50': 'launch_speed'})
preds_la50_ev50 = model.predict(df_pandas_predict_la50_ev50[feats])

df_pandas_predict_la50_ev75 = df_pandas_predict[['xLA_50', 'xEV_75']]
df_pandas_predict_la50_ev75 = df_pandas_predict_la50_ev75.rename(columns = {'xLA_50': 'launch_angle', 'xEV_75': 'launch_speed'})
preds_la50_ev75 = model.predict(df_pandas_predict_la50_ev75[feats])

# 75th percentile launch angles
df_pandas_predict_la75_ev25 = df_pandas_predict[['xLA_75', 'xEV_25']]
df_pandas_predict_la75_ev25 = df_pandas_predict_la75_ev25.rename(columns = {'xLA_75': 'launch_angle', 'xEV_25': 'launch_speed'})
preds_la75_ev25 = model.predict(df_pandas_predict_la75_ev25[feats])

df_pandas_predict_la75_ev50 = df_pandas_predict[['xLA_75', 'xEV_50']]
df_pandas_predict_la75_ev50 = df_pandas_predict_la75_ev50.rename(columns = {'xLA_75': 'launch_angle', 'xEV_50': 'launch_speed'})
preds_la75_ev50 = model.predict(df_pandas_predict_la75_ev50[feats])

df_pandas_predict_la75_ev75 = df_pandas_predict[['xLA_75', 'xEV_75']]
df_pandas_predict_la75_ev75 = df_pandas_predict_la75_ev75.rename(columns = {'xLA_75': 'launch_angle', 'xEV_75': 'launch_speed'})
preds_la75_ev75 = model.predict(df_pandas_predict_la75_ev75[feats])

# concat to df
df_bayes_ev = df_bayes_ev.with_columns(pl.Series("xRV_la25_ev25", preds_la25_ev25),
                             pl.Series("xRV_la25_ev50", preds_la25_ev50),
                             pl.Series("xRV_la25_ev75", preds_la25_ev75),

                             pl.Series("xRV_la50_ev25", preds_la50_ev25),
                             pl.Series("xRV_la50_ev50", preds_la50_ev50),
                             pl.Series("xRV_la50_ev75", preds_la50_ev75),

                             pl.Series("xRV_la75_ev25", preds_la75_ev25),
                             pl.Series("xRV_la75_ev50", preds_la75_ev50),
                             pl.Series("xRV_la75_ev75", preds_la75_ev75))

# Model 2 Preds

In [ ]:
df = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_model1.csv')
df_bayes_ev = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/df_bayes_ev.csv')

In [ ]:
# joining dfs
cols = ['game_year', 'player_name', 'pitch_type', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'pfx_x', 'pfx_z', 'spin_axis', 'release_speed', 'release_extension', 'arm_angle', 'release_spin_rate', 'ax', 'ay', 'az']
filter_cols = ['called_strike', 'called_ball', 'called_hbp', 'whiff']
df_join = df.select(cols + filter_cols)

df_bayes_ev = df_bayes_ev.join(df_join, on = cols, how = 'left')

In [ ]:
# getting non foul preds
df_bayes_ev = df_bayes_ev.with_columns(
    no_foul_pred_nn = 1 - pl.col('foul_pred_nn')
)

# getting event avg run values
called_strike = df_bayes_ev.filter(pl.col('called_strike') == 1)['delta_run_exp'].mean()
called_ball = df_bayes_ev.filter(pl.col('called_ball') == 1)['delta_run_exp'].mean()
called_hbp = df_bayes_ev.filter(pl.col('called_hbp') == 1)['delta_run_exp'].mean()

whiff = df_bayes_ev.filter(pl.col('whiff') == 1)['delta_run_exp'].mean()

foul = df_bayes_ev.filter(pl.col('foul') == 1)['delta_run_exp'].mean()

In [ ]:
# generating preds
df_bayes_ev = df_bayes_ev.with_columns(
    xRV_model2 =
    pl.col('take_pred_xgb')*pl.col('called_strike_pred_nn')*called_strike +
    pl.col('take_pred_xgb')*pl.col('called_ball_pred_nn')*called_ball +
    pl.col('take_pred_xgb')*pl.col('called_hbp_pred_nn')*called_hbp +

    pl.col('swing_pred_xgb')*pl.col('whiff_pred_xgb')*whiff +

    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('foul_pred_nn')*foul +

    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('no_foul_pred_nn')*0.25*0.25*pl.col('xRV_la25_ev25') +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('no_foul_pred_nn')*0.25*0.50*pl.col('xRV_la25_ev50') +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('no_foul_pred_nn')*0.25*0.25*pl.col('xRV_la25_ev75') +

    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('no_foul_pred_nn')*0.50*0.25*pl.col('xRV_la50_ev25') +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('no_foul_pred_nn')*0.50*0.50*pl.col('xRV_la50_ev50') +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('no_foul_pred_nn')*0.50*0.25*pl.col('xRV_la50_ev75') +

    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('no_foul_pred_nn')*0.25*0.25*pl.col('xRV_la75_ev25') +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('no_foul_pred_nn')*0.25*0.50*pl.col('xRV_la75_ev50') +
    pl.col('swing_pred_xgb')*pl.col('contact_pred_xgb')*pl.col('no_foul_pred_nn')*0.25*0.25*pl.col('xRV_la75_ev75')
)

In [ ]:
# model 2 pred distribution
sns.histplot(df_bayes_ev.select(pl.col('xRV_model2')))
plt.show()

In [ ]:
# model 2 best by year
df_bayes_ev.group_by(pl.col('game_year'), pl.col('player_name')).agg(pl.col('xRV_model2').mean()).sort('xRV_model2', descending = False).head(10)

In [ ]:
# model 2 best pitches by year
 (df_bayes_ev
    .group_by(pl.col('game_year'), pl.col('player_name'), pl.col('pitch_type'))
    .agg(pl.col('xRV_model2').mean())
    .sort('xRV_model2', descending = False).head(10)
)

In [ ]:
df_corr = pl.read_csv('/content/drive/MyDrive/Datasets/stuff/corr_df.csv')

In [ ]:
# regenerating baseline preds on filtered dataset
feats = ['righty', 'release_pos_x', 'release_pos_z', 'release_extension', 'arm_angle', 'release_speed', 'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'secondary_velo_differential', 'secondary_vert_differential', 'secondary_horz_differential']
target = 'delta_run_exp'

df_pandas = df_bayes_ev.to_pandas()
X = df_pandas[feats]
y = df_pandas[target]
model = XGBRegressor()
preds = cross_val_predict(model, X, y , cv=5, method='predict')

df_bayes_ev = df_bayes_ev.with_columns(pl.Series("xRV_baseline", preds))

In [ ]:
# putting on stuff+ scale
df_bayes_ev = df_bayes_ev.with_columns([
    (-((pl.col("xRV_baseline") - pl.col("xRV_baseline").mean()) / pl.col("xRV_baseline").std()) * 10 + 100).alias("Baseline_Stuff+"),
    (-((pl.col("xRV_model1") - pl.col("xRV_model1").mean()) / pl.col("xRV_model1").std()) * 10 + 100).alias("Model1_Stuff+"),
    (-((pl.col("xRV_model2") - pl.col("xRV_model2").mean()) / pl.col("xRV_model2").std()) * 10 + 100).alias("Model2_Stuff+")
])

In [ ]:
df_bayes_ev.write_csv('df_bayes_ev.csv')
files.download('df_bayes_ev.csv')

In [ ]:
# model 2 worst by year
df_bayes_ev.group_by(pl.col('game_year'), pl.col('player_name')).agg(pl.col('Model2_Stuff+').mean()).sort('Model2_Stuff+', descending = False).head(10)

In [ ]:
(df_bayes_ev
    .filter(pl.col('player_name') == 'Skenes, Paul')
    .group_by(pl.col('game_year'), pl.col('player_name'), pl.col('pitch_type'))
    .agg(pl.col('Model2_Stuff+').mean())
    .sort('Model2_Stuff+', descending = True).head(10)
)

In [ ]:
# model 2 best pitches by year
 (df_bayes_ev
    .group_by(pl.col('game_year'), pl.col('player_name'), pl.col('pitch_type'))
    .agg(pl.col('Model2_Stuff+').mean())
    .sort('Model2_Stuff+', descending = True).head(10)
)

In [ ]:
# model 1 best pitches by year
 (df_bayes_ev
    .group_by(pl.col('game_year'), pl.col('player_name'), pl.col('pitch_type'))
    .agg(pl.col('Model1_Stuff+').mean())
    .sort('Model1_Stuff+', descending = True).head(10)
)

In [ ]:
# baseline best pitches by year
 (df_bayes_ev
    .group_by(pl.col('game_year'), pl.col('player_name'), pl.col('pitch_type'))
    .agg(pl.col('Baseline_Stuff+').mean())
    .sort('Baseline_Stuff+', descending = True).head(10)
)

In [ ]:
# correlations for filtered dataset

# get average stuff for each pitcher
mean_stuff = df_bayes_ev.group_by(pl.col('game_year'), pl.col('player_name')).agg(pl.col('Baseline_Stuff+').mean())
mean_stuff1 = df_bayes_ev.group_by(pl.col('game_year'), pl.col('player_name')).agg(pl.col('Model1_Stuff+').mean())
mean_stuff2 = df_bayes_ev.group_by(pl.col('game_year'), pl.col('player_name')).agg(pl.col('Model2_Stuff+').mean())
# join to corr df
df_corr = df_corr.join(mean_stuff, left_on = ['Season', 'names'], right_on = ['game_year', 'player_name'], how='inner')
df_corr = df_corr.join(mean_stuff1, left_on = ['Season', 'names'], right_on = ['game_year', 'player_name'], how='inner')
df_corr = df_corr.join(mean_stuff2, left_on = ['Season', 'names'], right_on = ['game_year', 'player_name'], how='inner')
# rename for clarity
df_corr = df_corr.rename({'Stuff+': 'Fangraphs_Stuff+'})

# create corr matrix
y_vars = ['ERA_next', 'FIP_next', 'xFIP_next', 'SIERA_next']
x_vars = ['ERA', 'FIP', 'xFIP', 'SIERA', 'botStf', 'Fangraphs_Stuff+', 'Baseline_Stuff+', 'Model1_Stuff+', 'Model2_Stuff+']

correlations = []

for x in x_vars:
    row = []
    for y in y_vars:
        corr = df_corr.select(pl.corr(x, y)).item()
        row.append(abs(corr))
    correlations.append(row)

corr_pd = pd.DataFrame((correlations), index=x_vars, columns=y_vars)

# plot corr matrix
sns.heatmap(corr_pd, annot=True, vmin = 0.2, vmax = 0.65, cmap="coolwarm", fmt = '.2f')
plt.title(f"Stuff+ Correlation Matrix (Min 100 IP)")
plt.show()